In [ ]:
%%local
import numpy as np
import pandas as pd
from highcharts import Highchart, Highstock
from ipywidgets import widgets
from IPython.display import Javascript, display
import folium
from folium.plugins import MarkerCluster, FastMarkerCluster, HeatMap
import pydoop.hdfs as hdfs
from datetime import datetime
from dateutil.parser import parse
from termcolor import colored
import qgrid
import json

project_name = ""
project_user = ""

hdfs_prefix = "hdfs://"
hdfs_root_dir = '/Projects/{}'.format(project_name)
hdfs_current_dir = '/Projects/{}'.format(project_name)
hdfs_file_to_read = ''
fs = hdfs.hdfs(user='{}__{}'.format(project_name, project_user))
lfs = hdfs.hdfs("", 0)

def get_directory_contents():
    try:
        contents = []
        
        items = hdfs.hdfs.list_directory(fs, '{}{}'.format(hdfs_prefix, hdfs_current_dir))
    
        for item in items:
            contents.append('{} --- {}'.format(item['name'][item['name'].rfind('/') + 1:], item['kind']))
            
        return contents
    except:
        print(colored("An exception occured while trying to list contents.\nPlease make sure that you have set up correctly the username and project name.", "red", attrs=['bold']))
        return None

def open_file_or_directory(b):
    global hdfs_current_dir, hdfs_file_to_read
    
    try:
        file_or_directory = dropdown.value
        contents = file_or_directory.split('---')

        if contents[1].strip() == 'directory':
            display(Javascript("Jupyter.notebook.execute_cells([1])"))
            hdfs_current_dir = hdfs_current_dir + '/' + contents[0].strip()
        elif contents[1].strip() == 'file':
            hdfs_file_to_read = hdfs_current_dir + '/' + contents[0].strip()
            display(Javascript("Jupyter.notebook.execute_cells([2, 3])"))
    except:
        print(colored("An exception occured while trying to list contents.\nPlease make sure that you have set up correctly the username and project name.", "red", attrs=['bold']))

def get_geo_json_file():
    global hdfs_current_dir
    
    return hdfs_current_dir + '/' + geo_json_dropdown_var.value.split('---')[0]
        
def back_to_top(b):
    global hdfs_current_dir, hdfs_root_dir
    hdfs_current_dir = hdfs_root_dir
    display(Javascript("Jupyter.notebook.execute_cells([1])"))
    
def parameters_button_clicked(b):
    cell = dashboard_picker.value
    command = "Jupyter.notebook.execute_cells([{}])".format(5 + cell)
    
    display(Javascript(command))

def datetime_to_epoch(value):
    d = parse(value)
    epoch = (d - datetime(1970, 1, 1)).total_seconds()
    return epoch
    
def get_visualization_data(visualization_type, df):
    try:
        visualization_data = []
        categories = []
        chart_options = {'zoomType': 'xy'}
        plot_options = {}
        tooltip = {'enabled': True}
        legend_options = {}
        x_axis_options = {'title': {'text': x_axis_name.value}}
        y_axis_options = {'title': {'text': y_axis_name.value}}

        json_data = df.to_dict(orient='records')

        # Configuration options for scatter plot
        if visualization_type == 'scatter' or visualization_type == 'line' or visualization_type == 'area':
            if not class_var.value == None:
                temp_df = df.groupby(class_var.value)[[x_axis_var.value, y_axis_var.value]].apply(lambda x: x.values.tolist())
                categories = temp_df.index.tolist()
                visualization_data = temp_df.values.tolist()
            else:
                visualization_data = df[[x_axis_var.value, y_axis_var.value]].values.tolist()

            tooltip = {
                'headerFormat': '',
                'pointFormat': '{point.x}, {point.y}'
            }

            if x_axis_as_category_var.value:
                x_axis_options['type'] = 'category'

        # Configuration options for pie chart
        elif visualization_type == "pie":
            pie = df.groupby(class_var.value)[class_var.value].count().to_dict()

            for key, value in pie.items():
                visualization_data.append({'className': key, 'y': value, 'name': key})

            visualization_data = sorted(visualization_data, key=lambda x: x['y'], reverse=True)
            plot_options['pie'] = {'allowPointSelect': True, 'cursor': 'pointer'}
            tooltip = {
                'headerFormat': '',
                'pointFormat': '<b>{point.name}</b>: {point.percentage}%'
            }

            if show_in_legend.value:
                plot_options['pie']['showInLegend'] = True
                legend_options['align'] = 'right'
                legend_options['layout'] = 'vertical'
                legend_options['verticalAlign'] = 'top'

        # Configuration options for bar chart
        elif visualization_type == 'bar':

            if stacked_by.value is None:
                temp_df = df.groupby(class_var.value)[class_var.value].count().to_dict()

                for key, value in temp_df.items():
                    categories.append(key)
                    visualization_data.append({'name': key, 'y': int(value)})

    #             visualization_data = sorted(visualization_data, key=lambda x: x['y'], reverse=True)
            else:
                temp_df = df.groupby([class_var.value, stacked_by.value])[[class_var.value]].count()
                categories = temp_df[class_var.value].keys().levels[0].values.tolist()
                stacks = temp_df[class_var.value].keys().levels[1].values.tolist()

                for s in stacks:
                    d = []
                    for c in categories:
                        if s in temp_df[class_var.value][c]:
                            d.append(temp_df[class_var.value][c][s])
                        else:
                            d.append(0)
                    visualization_data.append({'type': 'column', 'data': d, 'name': s})

            tooltip = {
                'headerFormat': '',
                'pointFormat': '<b>{point.y}</b>'
            }

            x_axis_options['categories'] = categories


        # Configuration options for boxplot
        elif visualization_type == 'boxplot':
            #temp_df = df.groupby(class_var.value)[x_axis_var.value].apply(list)
            #categories = temp_df.index.tolist()
            #visualization_data = temp_df.apply(lambda x: np.sort(x).tolist()).values.tolist()

            temp_df = df.groupby(class_var.value)[x_axis_var.value].describe()
            for idx in temp_df.index:
                categories.append(idx)
                visualization_data.append({
                    'low': temp_df.loc[idx]['min'],
                    'q1': temp_df.loc[idx]['25%'],
                    'median': temp_df.loc[idx]['50%'],
                    'q3': temp_df.loc[idx]['75%'],
                    'high': temp_df.loc[idx]['max'],
                    'name': idx,
                })

            tooltip = {
                'headerFormat': '',
            }

            x_axis_options['categories'] = categories

        # Configuration options for histogram
        elif visualization_type == 'histogram':
            temp_df = df.groupby(pd.cut(df[x_axis_var.value], bins=histogram_bins.value, precision=2)).size()

            for item in temp_df.items():
                visualization_data.append({'x': item[0].left, 'y': int(item[1]), 'name': str(item[0])})

            tooltip = {
                'headerFormat': '',
                'pointFormat': '<b>{point.name}:</b> {point.y}'
            }

        # Configuration options for bubble chart
        elif visualization_type == "bubble":
            if not class_var.value == None:
                temp_df = df.groupby(class_var.value)[[x_axis_var.value, y_axis_var.value, z_axis_var.value]].apply(lambda x: x.values.tolist())
                categories = temp_df.index.tolist()
                visualization_data = temp_df.values.tolist()
            else:
                visualization_data = df[[x_axis_var.value, y_axis_var.value, z_axis_var.value]].values.tolist()

            tooltip = {
                'headerFormat': '',
                'pointFormat': '{point.x}, {point.y}, {point.z}'
            }

        # Configuration options for heatmap
        elif visualization_type == 'heatmap':
            if aggregation_function.value == 'max':
                temp_df = df.groupby([x_axis_var.value, y_axis_var.value])[z_axis_var.value].max().unstack(fill_value=0).stack()
            elif aggregation_function.value == 'min':
                temp_df = df.groupby([x_axis_var.value, y_axis_var.value])[z_axis_var.value].min().unstack(fill_value=0).stack()
            elif aggregation_function.value == 'average':
                temp_df = df.groupby([x_axis_var.value, y_axis_var.value])[z_axis_var.value].mean().unstack(fill_value=0).stack()
            elif aggregation_function.value == 'count':
                temp_df = df.groupby([x_axis_var.value, y_axis_var.value])[z_axis_var.value].count().unstack(fill_value=0).stack()
            elif aggregation_function.value == 'sum':
                temp_df = df.groupby([x_axis_var.value, y_axis_var.value])[z_axis_var.value].sum().unstack(fill_value=0).stack()

            cat_x = []
            cat_y = []

            for item in temp_df.index.levels[0]:
                cat_x.append(item)

            for item in temp_df.index.levels[1]:
                cat_y.append(item)

            x_axis_options['categories'] = cat_x
            y_axis_options['categories'] = cat_y
            chart_options['type'] = 'heatmap'
            chart_options['plotBorderWidth'] = 2
            tooltip = {
                'formatter': "function () {" + 
                    "return this.series.xAxis.categories[this.point.x] + '<br>' + this.series.yAxis.categories[this.point.y] + '<br>' + this.point.value;}"
            }

            for i in range(len(cat_x)):
                for j in range(len(cat_y)):
                    visualization_data.append([i, j, float(temp_df.values[len(cat_y) * i + j])])

        # Configuration options for timeseries
        elif visualization_type == 'time':
            temp_df = df.copy()

            if x_axis_as_timestamp_var.value:
                temp_df[x_axis_var.value] = temp_df[x_axis_var.value].apply(datetime_to_epoch) * 1000

            temp_df.sort_values(by=x_axis_var.value, inplace=True)

            if not class_var.value == None:
                temp_df_grouped = temp_df.groupby(class_var.value)[[x_axis_var.value, y_axis_var.value]].apply(lambda x: x.values.tolist())
                categories = temp_df_grouped.index.tolist()
                visualization_data = temp_df_grouped.values.tolist()
            else:
                visualization_data = temp_df[[x_axis_var.value, y_axis_var.value]].values.tolist()

            chart_options['zoomType'] = 'x'
            tooltip['valueDecimals'] = 2
        
        # Configuration options for gauge
        elif visualization_type == 'spline':
            visualization_data = df[[x_axis_var.value, y_axis_var.value]].values.tolist()

            tooltip = {
                'headerFormat': '',
                'pointFormat': '{point.x}, {point.y}'
            }
            
            if x_axis_inverted.value == 'True':
                x_axis_options['reversed'] = True
            else:
                x_axis_options['reversed'] = False

        return visualization_data, chart_options, categories, plot_options, tooltip, x_axis_options, y_axis_options, legend_options
    except:
        print(colored("Error in creating chart data.\n", "red", attrs=['bold']))
        raise
        
def fill_chart(chart, visualization_type, plot_data, categories, has_class):
    """
    Function for filling up a chart's data in the right way, based on the visualization type
    """
    try:
        if visualization_type == 'scatter' or visualization_type == 'line' or visualization_type == 'bubble' or visualization_type == 'area':        
            if has_class:
                for i in range(len(categories)):
                    chart.add_data_set(plot_data[i], visualization_type, str(categories[i]), marker={'enabled': True, 'symbol': 'circle'})
            else:
                chart.add_data_set(plot_data, visualization_type, categories, marker={'enabled': True, 'symbol': 'circle'})

        elif visualization_type == 'spline':
            chart.add_data_set(plot_data, visualization_type, categories, marker={'enabled': False})
                
        elif visualization_type == 'pie' or visualization_type == 'bar' or visualization_type == 'boxplot' or visualization_type == 'heatmap':
            if visualization_type == 'heatmap':
                chart.set_options('colorAxis', {'minColor': '#FFFFFF', 'maxColor': '#7CB5EC'})

            if visualization_type == 'bar' and not stacked_by.value == None:
                for item in plot_data:
                    chart.add_data_set(item['data'], 'column', item['name'])
            else:
                chart.add_data_set(plot_data, visualization_type, None)


        elif visualization_type == 'histogram':
            chart.add_data_set(plot_data, 'column', None, pointPadding=0, groupPadding=0, pointPlacement='between')

        elif visualization_type == 'time':        
            if has_class:
                for i in range(len(categories)):
                    chart.add_data_set(plot_data[i], 'line', str(categories[i]))
            else:
                chart.add_data_set(plot_data, 'line', categories)
        
        elif visualization_type == 'gauge':
            chart.add_data_set(plot_data, 'solidgauge', None)
    except:
        print(colored("Error while adding data to chart.\n", "red", attrs=['bold']))
        raise

def add_markers(map_obj, df):
    try:
        if markers_colors.value is not None and markers_text.value is not None:
            points = df[[latitude_var.value, longitude_var.value, markers_colors.value, markers_text.value]].values.tolist()
        elif markers_colors.value is not None:
            points = df[[latitude_var.value, longitude_var.value, markers_colors.value]].values.tolist()
        elif markers_text.value is not None:
            points = df[[latitude_var.value, longitude_var.value, markers_text.value]].values.tolist()
        else:
            points = df[[latitude_var.value, longitude_var.value]].values.tolist()

        for point in points:
            loc = (point[0], point[1])

            if markers_colors.value is not None:
                icon = folium.Icon(color=point[2], icon='map-marker')
            else:
                icon = None

            if markers_text.value is not None:
                if markers_colors.value is not None:
                    pop_up = point[3]
                else:
                    pop_up = point[2]
            else:
                pop_up = '({}, {})'.format(point[0], point[1])

            folium.Marker(location=loc, popup=pop_up, icon=icon).add_to(map_obj)
    except:
        print(colored("Error in adding markers on map.\n", "red", attrs=['bold']))
        raise
    
def create_marker_cluster(map_obj, df):
    try:
        points = df[[latitude_var.value, longitude_var.value]].values.tolist()

        if markers_colors.value is not None:
            icons = [folium.Icon(color=c[0], icon='map-marker') for c in df[[markers_colors.value]].values.tolist()]
        else:
            icons = None

        if markers_text.value is not None:
            popups = [text[0] for text in df[[markers_text.value]].values.tolist()]
        else:
            popups = ['({}, {})'.format(point[0], point[1]) for point in points]

        MarkerCluster(locations=points, popups=popups, icons=icons).add_to(map_obj)
    except:
        print(colored("Error while creating marker cluster.\n", "red", attrs=['bold']))
        raise
    
def create_fast_marker_cluster(map_obj, df):
    try:
        points = df[[latitude_var.value, longitude_var.value]].values.tolist()

        if markers_colors.value is not None:
            icons = [folium.Icon(color=c[0], icon='map-marker') for c in df[[markers_colors.value]].values.tolist()]
        else:
            icons = None

        if markers_text.value is not None:
            popups = [text[0] for text in df[[markers_text.value]].values.tolist()]
        else:
            popups = ['({}, {})'.format(point[0], point[1]) for point in points]

        callback = """\
            function (row) {
                var icon, marker;
                icon = L.AwesomeMarkers.icon({
                    icon: "map-marker", 
                    markerColor: "blue"
                });
                marker = L.marker(new L.LatLng(row[0], row[1]));
                marker.setIcon(icon);
                return marker;
            };
        """
        FastMarkerCluster(points, callback=callback).add_to(map_obj)
    except:
        print(colored("Error while creating fast marker cluster.\n", "red", attrs=['bold']))
        raise
        
def draw_route(map_obj, df):
    try:
        points = df[[latitude_var.value, longitude_var.value]].values.tolist()
        folium.Marker(points[0], popup='Start').add_to(map_obj)
        folium.Marker(points[-1], popup='End').add_to(map_obj)
        folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(map_obj)
    except:
        print(colored("Error while creating route.\n", "red", attrs=['bold']))
        raise
    
def draw_map_heatmap(map_obj, df):
    try:
        points = df[[latitude_var.value, longitude_var.value, heatmap_density_var.value]].values.tolist()
        max_density = df[heatmap_density_var.value].max()
        HeatMap(points, min_opacity=heatmap_opacity_var.value, radius=heatmap_point_radius_var.value, max_val=max_density).add_to(map_obj)
    except:
        print(colored("Error while creating heatmap.\n", "red", attrs=['bold']))
        raise
    
def create_chart(df, visualization_type):
    try:
        if visualization_type == 'grid':
            return qgrid.show_grid(df)

        elif visualization_type == 'map':
            folium_map = folium.Map(location=[df.loc[0][latitude_var.value], df.loc[0][longitude_var.value]])

            if map_type_var.value == 'Markers':
                add_markers(folium_map, df)
            elif map_type_var.value == 'MarkerCluster':
                create_marker_cluster(folium_map, df)
            elif map_type_var.value == 'FastMarkerCluster':
                create_fast_marker_cluster(folium_map, df)
            elif map_type_var.value == 'Route':
                draw_route(folium_map, df)
            elif map_type_var.value == 'Heatmap':
                draw_map_heatmap(folium_map, df)
            elif map_type_var.value == 'Route & Markers':
                df_route = df[df.category == 'route']
                df_markers = df[df.category == 'marker']
                draw_route(folium_map, df_route)
                add_markers(folium_map, df_markers)

            return folium_map

        elif visualization_type == 'choropleth':
            geo_json = json.load(fs.open_file(get_geo_json_file()))

            folium_map = folium.Map(location=[48, -102], zoom_start=3)

            folium.Choropleth(
                geo_data=geo_json,
                name='choropleth',
                data=df,
                columns=[choropleth_key_column_var.value, choropleth_value_column_var.value],
                key_on=choropleth_join_column_var.value,
                fill_color=choropleth_colorscale_var.value,
                fill_opacity=choropleth_fill_opacity_var.value,
                line_opacity=choropleth_line_opacity_var.value,
                legend_name=choropleth_legend_var.value
            ).add_to(folium_map)

            folium.LayerControl().add_to(folium_map)

            return folium_map

        elif visualization_type == 'time':
            plot_data, chart_options, categories, plot_options, tooltip_options, x_axis_options, \
                y_axis_options, legend_options = get_visualization_data(visualization_type, df)
            chart = Highstock()

            options = {
                'chart': chart_options,
                'title' : { 
                    'text' : plot_title.value 
                },
                'subtitle': {
                    'text': plot_subtitle.value 
                },
                'tooltip': tooltip_options
            }

            chart.set_dict_options(options)

            fill_chart(chart, visualization_type, plot_data, categories, not class_var.value == None)

            return chart

        else:
            chart = Highchart()

            plot_data, chart_options, categories, plot_options, tooltip_options, x_axis_options, \
                y_axis_options, legend_options = get_visualization_data(visualization_type, df)

            if len(categories) == 0:
                categories = None

            chart.set_options('chart', chart_options)

            options = {
                'title': {
                    'text': plot_title.value
                },

                'subtitle': {
                    'text': plot_subtitle.value
                },

                'xAxis': x_axis_options,

                'yAxis': y_axis_options,

                'legend': legend_options,

                'tooltip': tooltip_options,

                'plotOptions': plot_options,

                'chart': {
                    'width': 950,
                    'height': 550
                }
            }

            chart.set_dict_options(options)
            fill_chart(chart, visualization_type, plot_data, categories, not class_var.value == None)

            return chart
    except:
        print(colored("An exception occured while trying to create chart.\nPlease make sure that you have correctly set up all mandatory parameters.", "red", attrs=['bold']))
        return None

def save_chart(chart, name):
    try:
        if "highcharts" in str(type(chart)):
            chart.save_file(name)
        else:
            chart.save(outfile=name + '.html')
        lfs.copy(name + '.html', fs, hdfs_current_dir)
        
        return True
    except:
        return False

Javascript("Jupyter.notebook.execute_cells([1])")

In [ ]:
%%local
dirs = get_directory_contents()

if dirs is not None:
    current_directory = widgets.HTML(value = '<h4>Current Directory: <b>{}</b></h4>'.format(hdfs_current_dir))
    dropdown = widgets.Dropdown(options=dirs, value=None, description='Folder/File:', style={'description_width': 'initial'})

    delimiter_dropdown = widgets.Dropdown(options=[',', ';', 'TAB'], value=',', description='CSV Delimiter:', style={'description_width': 'initial'})

    open_button = widgets.Button(description='Open', button_style="primary")
    open_button.on_click(open_file_or_directory)

    back_to_top_button = widgets.Button(description="Back to Root", button_style="primary")
    back_to_top_button.on_click(back_to_top)

    vbox = widgets.VBox([current_directory, dropdown, delimiter_dropdown, widgets.HBox([open_button, back_to_top_button])])

    display(vbox)

In [ ]:
%%local

try:
    df = pd.read_csv(fs.open_file(hdfs_file_to_read), delimiter=delimiter_dropdown.value)
    print(df.head(15))
except:
    print(colored("An exception occured while trying to read CSV file.", "red", attrs=['bold']))

In [ ]:
%%HTML
<script>
    function visualizationChange(element) {
        var ids = ['scatter', 'pie', 'bar', 'line', 'boxplot', 'histogram', 'time', 'heatmap', 'bubble', 'map', 'grid', 'area', 'spline', 'choropleth'];
        
        for (var i = 0; i < ids.length; i++) {
            if (element.id == ids[i]) {
                //IPython.notebook.kernel.execute("visualization_type='" + ids[i] + "'");
                var nb = Jupyter.notebook;
                var cell = nb.get_cell(4);
                cell.set_text("%%local\nvisualization_type='" + ids[i] + "'");
                cell.execute();
            } else {
                $("#" + ids[i]).prop("checked", false);
            }
        }
    };
</script>

<div id="plotSelector">
    <h2>Select a visualization type</h2>
    
    </br>
    
    <div class="row">
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="scatter" id="scatter" onClick="visualizationChange(this)">Scatter Plot</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTguMS4xLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4PSIwcHgiIHk9IjBweCIgdmlld0JveD0iMCAwIDE3LjQ1NSAxNy40NTUiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDE3LjQ1NSAxNy40NTU7IiB4bWw6c3BhY2U9InByZXNlcnZlIiB3aWR0aD0iMzJweCIgaGVpZ2h0PSIzMnB4Ij4KPGc+Cgk8Zz4KCQk8cG9seWdvbiBwb2ludHM9IjEuNjU4LDE1LjY2NyAxLjY1OCwwLjEzIDAsMC4xMyAwLDE3LjMyNSAxNy40NTUsMTcuMzI1IDE3LjQ1NSwxNS42NjcgICAiIGZpbGw9IiMwMDAwMDAiLz4KCQk8Y2lyY2xlIGN4PSI1LjEzNCIgY3k9IjExLjg0MyIgcj0iMS41MTkiIGZpbGw9IiMwMDAwMDAiLz4KCQk8Y2lyY2xlIGN4PSI2LjE0NyIgY3k9IjcuNjMxIiByPSIxLjUxOSIgZmlsbD0iIzAwMDAwMCIvPgoJCTxjaXJjbGUgY3g9IjguMzMxIiBjeT0iMTAuNDk2IiByPSIxLjUyIiBmaWxsPSIjMDAwMDAwIi8+CgkJPGNpcmNsZSBjeD0iOS44NTEiIGN5PSI2LjU1NyIgcj0iMS41MTkiIGZpbGw9IiMwMDAwMDAiLz4KCQk8Y2lyY2xlIGN4PSIxMy44NTkiIGN5PSI2LjU1OCIgcj0iMS41MiIgZmlsbD0iIzAwMDAwMCIvPgoJCTxjaXJjbGUgY3g9IjExLjk3IiBjeT0iOS41OTUiIHI9IjEuNTIiIGZpbGw9IiMwMDAwMDAiLz4KCQk8Y2lyY2xlIGN4PSIxMS45NzEiIGN5PSIzLjUxOSIgcj0iMS41MTkiIGZpbGw9IiMwMDAwMDAiLz4KCTwvZz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8L3N2Zz4K" />
            </div>
        </div>
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="pie" id="pie" onClick="visualizationChange(this)">Pie Chart</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTYuMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjwhRE9DVFlQRSBzdmcgUFVCTElDICItLy9XM0MvL0RURCBTVkcgMS4xLy9FTiIgImh0dHA6Ly93d3cudzMub3JnL0dyYXBoaWNzL1NWRy8xLjEvRFREL3N2ZzExLmR0ZCI+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4PSIwcHgiIHk9IjBweCIgd2lkdGg9IjMycHgiIGhlaWdodD0iMzJweCIgdmlld0JveD0iMCAwIDk2LjQ5MSA5Ni40OTEiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDk2LjQ5MSA5Ni40OTE7IiB4bWw6c3BhY2U9InByZXNlcnZlIj4KPGc+Cgk8Zz4KCQk8cGF0aCBkPSJNNTIuNjM5LDAuNDYzdjQ1LjczMmw0MC40MzksMTUuMTA1YzIuMTk3LTUuMjI1LDMuNDEzLTEwLjk2MywzLjQxMy0xNi45ODVDOTYuNDkxLDIwLjA5Nyw3Ni44NTUsMC40NjMsNTIuNjM5LDAuNDYzeiIgZmlsbD0iIzAwMDAwMCIvPgoJCTxwYXRoIGQ9Ik00My44NTQsOC4zMjJDMTkuNjM1LDguMzIyLDAsMjcuOTU1LDAsNTIuMTc0YzAsMjQuMjIxLDE5LjYzNSw0My44NTQsNDMuODU0LDQzLjg1NGMxOC4xOTcsMCwzMy44MDMtMTEuMDg0LDQwLjQ0LTI2Ljg2OSAgICBsLTQwLjQ0LTE1LjEwNVY4LjMyMnoiIGZpbGw9IiMwMDAwMDAiLz4KCTwvZz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8L3N2Zz4K" />
            </div>
        </div>
    </div>
    
    </br>
    
    <div class="row">
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="bar" id="bar" onClick="visualizationChange(this)">Bar Chart</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTYuMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjwhRE9DVFlQRSBzdmcgUFVCTElDICItLy9XM0MvL0RURCBTVkcgMS4xLy9FTiIgImh0dHA6Ly93d3cudzMub3JnL0dyYXBoaWNzL1NWRy8xLjEvRFREL3N2ZzExLmR0ZCI+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4PSIwcHgiIHk9IjBweCIgd2lkdGg9IjMycHgiIGhlaWdodD0iMzJweCIgdmlld0JveD0iMCAwIDEwMC43ODcgMTAwLjc4NyIgc3R5bGU9ImVuYWJsZS1iYWNrZ3JvdW5kOm5ldyAwIDAgMTAwLjc4NyAxMDAuNzg3OyIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSI+CjxnPgoJPGc+CgkJPHBhdGggZD0iTTk5LjI4OCw5NC44OTRoLTcuMTIxVjkwLjA2YzAtMC4yNzUtMC4yMjQtMC41LTAuNS0wLjVzLTAuNSwwLjIyNS0wLjUsMC41djQuODM0aC05VjkwLjA2YzAtMC4yNzUtMC4yMjQtMC41LTAuNS0wLjUgICAgcy0wLjUsMC4yMjUtMC41LDAuNXY0LjgzNGgtOVY5MC4wNmMwLTAuMjc1LTAuMjI0LTAuNS0wLjUtMC41cy0wLjUsMC4yMjUtMC41LDAuNXY0LjgzNGgtOVY5MC4wNmMwLTAuMjc1LTAuMjI0LTAuNS0wLjUtMC41ICAgIHMtMC41LDAuMjI1LTAuNSwwLjV2NC44MzRoLTlWOTAuMDZjMC0wLjI3NS0wLjIyNC0wLjUtMC41LTAuNXMtMC41LDAuMjI1LTAuNSwwLjV2NC44MzRoLTlWOTAuMDZjMC0wLjI3NS0wLjIyNC0wLjUtMC41LTAuNSAgICBzLTAuNSwwLjIyNS0wLjUsMC41djQuODM0aC05VjkwLjA2YzAtMC4yNzUtMC4yMjQtMC41LTAuNS0wLjVzLTAuNSwwLjIyNS0wLjUsMC41djQuODM0aC05VjkwLjA2YzAtMC4yNzUtMC4yMjQtMC41LTAuNS0wLjUgICAgcy0wLjUsMC4yMjUtMC41LDAuNXY0LjgzNEgzVjgzLjU2MWg0LjAxNmMwLjI3NiwwLDAuNS0wLjIyNCwwLjUtMC41cy0wLjIyNC0wLjUtMC41LTAuNUgzdi05aDQuMDE2YzAuMjc2LDAsMC41LTAuMjI0LDAuNS0wLjUgICAgcy0wLjIyNC0wLjUtMC41LTAuNUgzdi05aDQuMDE2YzAuMjc2LDAsMC41LTAuMjI0LDAuNS0wLjVzLTAuMjI0LTAuNS0wLjUtMC41SDN2LTloNC4wMTZjMC4yNzYsMCwwLjUtMC4yMjQsMC41LTAuNSAgICBzLTAuMjI0LTAuNS0wLjUtMC41SDN2LTloNC4wMTZjMC4yNzYsMCwwLjUtMC4yMjQsMC41LTAuNXMtMC4yMjQtMC41LTAuNS0wLjVIM3YtOWg0LjAxNmMwLjI3NiwwLDAuNS0wLjIyNCwwLjUtMC41ICAgIHMtMC4yMjQtMC41LTAuNS0wLjVIM3YtOWg0LjAxNmMwLjI3NiwwLDAuNS0wLjIyNCwwLjUtMC41cy0wLjIyNC0wLjUtMC41LTAuNUgzdi05aDQuMDE2YzAuMjc2LDAsMC41LTAuMjI0LDAuNS0wLjUgICAgcy0wLjIyNC0wLjUtMC41LTAuNUgzVjQuMzk0YzAtMC44MjktMC42NzItMS41LTEuNS0xLjVTMCwzLjU2NCwwLDQuMzk0djkyYzAsMC44MjgsMC42NzIsMS41LDEuNSwxLjVoOTcuNzg3ICAgIGMwLjgyOCwwLDEuNS0wLjY3MiwxLjUtMS41UzEwMC4xMTYsOTQuODk0LDk5LjI4OCw5NC44OTR6IiBmaWxsPSIjMDAwMDAwIi8+CgkJPHJlY3QgeD0iMTUuNjQ0IiB5PSIxNi40OTciIHdpZHRoPSIxMC4yNSIgaGVpZ2h0PSI2Ny43OTUiIGZpbGw9IiMwMDAwMDAiLz4KCQk8cmVjdCB4PSIzNy4zOTQiIHk9IjQyLjc5MSIgd2lkdGg9IjEwLjI1IiBoZWlnaHQ9IjQxLjUiIGZpbGw9IiMwMDAwMDAiLz4KCQk8cmVjdCB4PSI1OC44OTUiIHk9IjI4LjU0MSIgd2lkdGg9IjEwLjI1IiBoZWlnaHQ9IjU1Ljc1IiBmaWxsPSIjMDAwMDAwIi8+CgkJPHJlY3QgeD0iODEuMzk1IiB5PSIzNS4yOTEiIHdpZHRoPSIxMC4yNSIgaGVpZ2h0PSI0OSIgZmlsbD0iIzAwMDAwMCIvPgoJPC9nPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+Cjwvc3ZnPgo=" />
            </div>
        </div>
        <div class="col col-md-6">
             <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="line" id="line" onClick="visualizationChange(this)">Line Chart</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTkuMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4PSIwcHgiIHk9IjBweCIgdmlld0JveD0iMCAwIDM3Ni4xNjMgMzc2LjE2MyIgc3R5bGU9ImVuYWJsZS1iYWNrZ3JvdW5kOm5ldyAwIDAgMzc2LjE2MyAzNzYuMTYzOyIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSIgd2lkdGg9IjMycHgiIGhlaWdodD0iMzJweCI+CjxnPgoJPGc+CgkJPGc+CgkJCTxwYXRoIGQ9Ik0zNDkuNTE4LDM0Ni45MDZINDEuNzk2Yy0xMS41NDIsMC0yMC44OTgtOS4zNTYtMjAuODk4LTIwLjg5OHYtMzA3LjJjMC01Ljc3MS00LjY3OC0xMC40NDktMTAuNDQ5LTEwLjQ0OSAgICAgUzAsMTMuMDM3LDAsMTguODA4djMwNy4yYzAsMjMuMDgzLDE4LjcxMyw0MS43OTYsNDEuNzk2LDQxLjc5NmgzMDcuNzIyYzUuNzcxLDAsMTAuNDQ5LTQuNjc4LDEwLjQ0OS0xMC40NDkgICAgIEMzNTkuOTY3LDM1MS41ODQsMzU1LjI4OSwzNDYuOTA2LDM0OS41MTgsMzQ2LjkwNnoiIGZpbGw9IiMwMDAwMDAiLz4KCQkJPHBvbHlnb24gcG9pbnRzPSIzMzYuOTgsNDQuNDA4IDMzNi45OCw0NC40MDggMzM2Ljk4LDQ0LjQwOCAgICAiIGZpbGw9IiMwMDAwMDAiLz4KCQkJPHBhdGggZD0iTTMzNi45OCw0NC40MDhjLTIxLjY0MSwwLTM5LjE4NCwxNy41NDMtMzkuMTg0LDM5LjE4NGMwLjAwMiw5Ljc4NSwzLjUyNiwxOS4yNDMsOS45MjcsMjYuNjQ1bC0yMi40NjUsMjYuNjQ1ICAgICBjLTUuMDktMi4zMTgtMTAuNjA0LTMuNTYzLTE2LjE5Ni0zLjY1N2MtMjEuNjQxLDAtMzkuMTg0LDE3LjU0My0zOS4xODQsMzkuMTg0djBjLTAuMTE0LDEuOTMxLDAuMDYyLDMuODY4LDAuNTIyLDUuNzQ3ICAgICBsLTI2LjEyMiw3LjMxNGMtMTAuMjc3LTE5LjA0NC0zNC4wNDctMjYuMTUyLTUzLjA5Mi0xNS44NzRjLTEyLjU3Miw2Ljc4NS0yMC40NTYsMTkuODc0LTIwLjU3NCwzNC4xNiAgICAgYzAuMDYyLDQuOTg2LDAuOTQ1LDkuOTI5LDIuNjEyLDE0LjYyOWwtMTkuODUzLDIwLjg5OGMtNi40OTktNC4zNDgtMTQuMTI0LTYuNzA5LTIxLjk0My02Ljc5MiAgICAgYy0yMS42NDEsMC0zOS4xODQsMTcuNTQzLTM5LjE4NCwzOS4xODRjMCwyMS42NDEsMTcuNTQzLDM5LjE4NCwzOS4xODQsMzkuMTg0YzIxLjY0MSwwLDM5LjE4NC0xNy41NDMsMzkuMTg0LTM5LjE4NCAgICAgYy0wLjE1MS01LjU4Ni0xLjM5NC0xMS4wODctMy42NTctMTYuMTk2bDE5LjMzMS0yMC4zNzZjNi44LDUuMDU0LDE1LjAzOCw3LjgsMjMuNTEsNy44MzcgICAgIGMyMC43NTUsMC4wMTcsMzcuODItMTYuMzU2LDM4LjY2MS0zNy4wOTRsMzAuMzAyLTguMzU5YzcuNDg4LDguOTksMTguNjAyLDE0LjE2NCwzMC4zMDIsMTQuMTA2ICAgICBjMjEuNTIzLTAuMjgxLDM4LjkwMi0xNy42NjEsMzkuMTg0LTM5LjE4NGMtMC4wMjEtNy44MjktMi4zODctMTUuNDcxLTYuNzkyLTIxLjk0M2wyNC41NTUtMjkuMjU3ICAgICBjMy41NDksMS4xMTEsNy4yNTQsMS42NCwxMC45NzEsMS41NjdjMjEuNjQxLDAsMzkuMTg0LTE3LjU0MywzOS4xODQtMzkuMTg0UzM1OC42Miw0NC40MDgsMzM2Ljk4LDQ0LjQwOHogTTkxLjQyOSwyODkuOTU5ICAgICBjLTEwLjA5OSwwLTE4LjI4Ni04LjE4Ny0xOC4yODYtMTguMjg2YzAtMTAuMDk5LDguMTg3LTE4LjI4NiwxOC4yODYtMTguMjg2czE4LjI4Niw4LjE4NywxOC4yODYsMTguMjg2ICAgICBDMTA5LjcxNCwyODEuNzcyLDEwMS41MjcsMjg5Ljk1OSw5MS40MjksMjg5Ljk1OXogTTE2OS43OTYsMjIyLjA0MWMtMTAuMDk5LDAtMTguMjg2LTguMTg3LTE4LjI4Ni0xOC4yODYgICAgIHM4LjE4Ny0xOC4yODYsMTguMjg2LTE4LjI4NmMxMC4wOTksMCwxOC4yODYsOC4xODcsMTguMjg2LDE4LjI4NlMxNzkuODk1LDIyMi4wNDEsMTY5Ljc5NiwyMjIuMDQxeiBNMjY5LjA2MSwxOTAuNjk0ICAgICBjLTEwLjA5OSwwLTE4LjI4Ni04LjE4Ny0xOC4yODYtMTguMjg2YzAtMTAuMDk5LDguMTg3LTE4LjI4NiwxOC4yODYtMTguMjg2YzEwLjA5OSwwLDE4LjI4Niw4LjE4NywxOC4yODYsMTguMjg2ICAgICBDMjg3LjM0NywxODIuNTA3LDI3OS4xNiwxOTAuNjk0LDI2OS4wNjEsMTkwLjY5NHogTTMzNi45OCwxMDEuODc4Yy0xMC4wOTksMC0xOC4yODYtOC4xODctMTguMjg2LTE4LjI4NiAgICAgczguMTg3LTE4LjI4NiwxOC4yODYtMTguMjg2czE4LjI4Niw4LjE4NywxOC4yODYsMTguMjg2UzM0Ny4wNzksMTAxLjg3OCwzMzYuOTgsMTAxLjg3OHoiIGZpbGw9IiMwMDAwMDAiLz4KCQk8L2c+Cgk8L2c+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPC9zdmc+Cg==" />
            </div>
        </div>
    </div>
    
    </br>
    
    <div class="row">
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="boxplot" id="boxplot" onClick="visualizationChange(this)">Box Plot</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTYuMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjwhRE9DVFlQRSBzdmcgUFVCTElDICItLy9XM0MvL0RURCBTVkcgMS4xLy9FTiIgImh0dHA6Ly93d3cudzMub3JnL0dyYXBoaWNzL1NWRy8xLjEvRFREL3N2ZzExLmR0ZCI+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4PSIwcHgiIHk9IjBweCIgd2lkdGg9IjMycHgiIGhlaWdodD0iMzJweCIgdmlld0JveD0iMCAwIDEwMC43ODcgMTAwLjc4NyIgc3R5bGU9ImVuYWJsZS1iYWNrZ3JvdW5kOm5ldyAwIDAgMTAwLjc4NyAxMDAuNzg3OyIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSI+CjxnPgoJPGc+CgkJPHBhdGggZD0iTTI2LjgzMiw1My4yOTlWNzUuODJjMCwwLjc5MywwLjY0MywxLjQzNiwxLjQzNiwxLjQzNnMxLjQzNi0wLjY0MywxLjQzNi0xLjQzNlY1My4yOTkgICAgYzIuMjA5LTAuMTg0LDMuOTQ1LTIuMDMxLDMuOTQ1LTQuMjg4VjM1LjYxN2MwLTIuMjU2LTEuNzM2LTQuMTA0LTMuOTQ1LTQuMjg3VjEyLjYzMmMwLTAuNzkyLTAuNjQzLTEuNDM1LTEuNDM2LTEuNDM1ICAgIHMtMS40MzYsMC42NDMtMS40MzYsMS40MzV2MTguNjk3Yy0yLjIwOSwwLjE4My0zLjk0NywyLjAzMS0zLjk0Nyw0LjI4N3YxMy4zOTVDMjIuODg0LDUxLjI2OCwyNC42MjMsNTMuMTE2LDI2LjgzMiw1My4yOTl6IiBmaWxsPSIjMDAwMDAwIi8+CgkJPHBhdGggZD0iTTU1LjUzNCw0My4yNTNWNzUuODJjMCwwLjc5MywwLjY0NCwxLjQzNiwxLjQzNywxLjQzNnMxLjQzNC0wLjY0MywxLjQzNC0xLjQzNlY0My4yNTMgICAgYzIuMjEyLTAuMTg0LDMuOTQ3LTIuMDMxLDMuOTQ3LTQuMjg3VjI1LjU3MWMwLTIuMjU3LTEuNzM1LTQuMTA0LTMuOTQ3LTQuMjg3di04LjY1MWMwLTAuNzkyLTAuNjQxLTEuNDM1LTEuNDM0LTEuNDM1ICAgIHMtMS40MzcsMC42NDMtMS40MzcsMS40MzV2OC42NTFjLTIuMjA5LDAuMTgzLTMuOTQ3LDIuMDMtMy45NDcsNC4yODd2MTMuMzk2QzUxLjU4Nyw0MS4yMjIsNTMuMzI1LDQzLjA3LDU1LjUzNCw0My4yNTN6IiBmaWxsPSIjMDAwMDAwIi8+CgkJPHBhdGggZD0iTTgyLjU2Myw1Ny4zNjZWNzUuODJjMCwwLjc5NCwwLjY0MywxLjQzOCwxLjQzNiwxLjQzOGMwLjc5MSwwLDEuNDM3LTAuNjQ0LDEuNDM3LTEuNDM4VjU3LjM2NiAgICBjMi4yMDctMC4xODMsMy45NDQtMi4wMzEsMy45NDQtNC4yODdWMzkuNjgzYzAtMi4yNTYtMS43MzctNC4xMDQtMy45NDQtNC4yODdWMTIuNjMyYzAtMC43OTItMC42NDYtMS40MzUtMS40MzctMS40MzUgICAgYy0wLjc5MywwLTEuNDM2LDAuNjQzLTEuNDM2LDEuNDM1djIyLjc2NGMtMi4yMTEsMC4xODQtMy45NDcsMi4wMzEtMy45NDcsNC4yODd2MTMuMzk2ICAgIEM3OC42MTYsNTUuMzM0LDgwLjM1Myw1Ny4xODQsODIuNTYzLDU3LjM2NnoiIGZpbGw9IiMwMDAwMDAiLz4KCQk8cGF0aCBkPSJNOTYuMzk0LDk3Ljc4N2gtOC4xNjZ2LTQuMDE2YzAtMC4yNzYtMC4yMjUtMC41LTAuNS0wLjVjLTAuMjc3LDAtMC41LDAuMjI0LTAuNSwwLjV2NC4wMTZoLTl2LTQuMDE2ICAgIGMwLTAuMjc2LTAuMjI1LTAuNS0wLjUtMC41Yy0wLjI3NywwLTAuNSwwLjIyNC0wLjUsMC41djQuMDE2aC05di00LjAxNmMwLTAuMjc2LTAuMjI1LTAuNS0wLjUtMC41Yy0wLjI3NywwLTAuNSwwLjIyNC0wLjUsMC41ICAgIHY0LjAxNmgtOXYtNC4wMTZjMC0wLjI3Ni0wLjIyNS0wLjUtMC41LTAuNWMtMC4yNzcsMC0wLjUsMC4yMjQtMC41LDAuNXY0LjAxNmgtOXYtNC4wMTZjMC0wLjI3Ni0wLjIyNS0wLjUtMC41LTAuNSAgICBjLTAuMjc3LDAtMC41LDAuMjI0LTAuNSwwLjV2NC4wMTZoLTl2LTQuMDE2YzAtMC4yNzYtMC4yMjUtMC41LTAuNS0wLjVjLTAuMjc3LDAtMC41LDAuMjI0LTAuNSwwLjV2NC4wMTZoLTl2LTQuMDE2ICAgIGMwLTAuMjc2LTAuMjI1LTAuNS0wLjUtMC41Yy0wLjI3NywwLTAuNSwwLjIyNC0wLjUsMC41djQuMDE2aC05di00LjAxNmMwLTAuMjc2LTAuMjI1LTAuNS0wLjUtMC41Yy0wLjI3NywwLTAuNSwwLjIyNC0wLjUsMC41ICAgIHY0LjAxNkg1Ljg5NFY3OS42MjFoNC44MzRjMC4yNzUsMCwwLjUtMC4yMjUsMC41LTAuNXMtMC4yMjUtMC41LTAuNS0wLjVINS44OTR2LTloNC44MzRjMC4yNzUsMCwwLjUtMC4yMjUsMC41LTAuNSAgICBzLTAuMjI1LTAuNS0wLjUtMC41SDUuODk0di05aDQuODM0YzAuMjc1LDAsMC41LTAuMjI1LDAuNS0wLjVzLTAuMjI1LTAuNS0wLjUtMC41SDUuODk0di05aDQuODM0YzAuMjc1LDAsMC41LTAuMjI0LDAuNS0wLjUgICAgcy0wLjIyNS0wLjUtMC41LTAuNUg1Ljg5NHYtOWg0LjgzNGMwLjI3NSwwLDAuNS0wLjIyNCwwLjUtMC41cy0wLjIyNS0wLjUtMC41LTAuNUg1Ljg5NHYtOWg0LjgzNGMwLjI3NSwwLDAuNS0wLjIyNCwwLjUtMC41ICAgIHMtMC4yMjUtMC41LTAuNS0wLjVINS44OTR2LTloNC44MzRjMC4yNzUsMCwwLjUtMC4yMjQsMC41LTAuNXMtMC4yMjUtMC41LTAuNS0wLjVINS44OTR2LTloNC44MzRjMC4yNzUsMCwwLjUtMC4yMjQsMC41LTAuNSAgICBzLTAuMjI1LTAuNS0wLjUtMC41SDUuODk0VjEuNWMwLTAuODI4LTAuNjcyLTEuNS0xLjUtMS41cy0xLjUsMC42NzItMS41LDEuNXY5Ny43ODdjMCwwLjgyOCwwLjY3MiwxLjUsMS41LDEuNWg5MiAgICBjMC44MjgsMCwxLjUtMC42NzIsMS41LTEuNVM5Ny4yMjIsOTcuNzg3LDk2LjM5NCw5Ny43ODd6IiBmaWxsPSIjMDAwMDAwIi8+Cgk8L2c+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPC9zdmc+Cg==" />
            </div>
        </div>
        <div class="col col-md-6">
             <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="histogram" id="histogram" onClick="visualizationChange(this)">Histogram</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTguMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjwhRE9DVFlQRSBzdmcgUFVCTElDICItLy9XM0MvL0RURCBTVkcgMS4xLy9FTiIgImh0dHA6Ly93d3cudzMub3JnL0dyYXBoaWNzL1NWRy8xLjEvRFREL3N2ZzExLmR0ZCI+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4PSIwcHgiIHk9IjBweCIgdmlld0JveD0iMCAwIDQwMCA0MDAiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDQwMCA0MDA7IiB4bWw6c3BhY2U9InByZXNlcnZlIiB3aWR0aD0iMzJweCIgaGVpZ2h0PSIzMnB4Ij4KPGc+Cgk8cGF0aCBkPSJNMzg0LjM3NSwyODUuMjI5SDk1LjgzNVY0Mi44ODFjMC04LjYyOS02Ljk5Ni0xNS42MjUtMTUuNjI1LTE1LjYyNWMtOC42MjgsMC0xNS42MjUsNi45OTYtMTUuNjI1LDE1LjYyNXYyNDIuMzQ5aC00OC45NiAgIEM2Ljk5NywyODUuMjI5LDAsMjkyLjIyNiwwLDMwMC44NTRjMCw4LjYyOCw2Ljk5NywxNS42MjUsMTUuNjI1LDE1LjYyNWg0OC45NnY0MC42NGMwLDguNjMxLDYuOTk3LDE1LjYyNSwxNS42MjUsMTUuNjI1ICAgYzguNjI5LDAsMTUuNjI1LTYuOTk0LDE1LjYyNS0xNS42MjV2LTQwLjY0aDI4OC41NGM4LjYyOCwwLDE1LjYyNS02Ljk5NywxNS42MjUtMTUuNjI1ICAgQzQwMCwyOTIuMjI2LDM5My4wMDMsMjg1LjIyOSwzODQuMzc1LDI4NS4yMjl6IiBmaWxsPSIjMDAwMDAwIi8+Cgk8cGF0aCBkPSJNMTI4LDI0OS4wOTZjMCwyLjc2MSwyLjIzOCw1LDUsNWg0M2MyLjc2MiwwLDUtMi4yMzksNS01VjEyMy4xMjFMMTI4LDE2NC41VjI0OS4wOTZ6IiBmaWxsPSIjMDAwMDAwIi8+Cgk8cGF0aCBkPSJNMjczLDI1NC4wOTZoNDNjMi43NjIsMCw1LTIuMjM5LDUtNVYxMDQuNzNsLTUzLDQxLjQxdjEwMi45NTVDMjY4LDI1MS44NTcsMjcwLjIzOCwyNTQuMDk2LDI3MywyNTQuMDk2eiIgZmlsbD0iIzAwMDAwMCIvPgoJPHBhdGggZD0iTTIzMS4wNjQsMTY0LjMzM2MtNy4yMjctMS4xMzUtMTMuNzI0LTUuMjYxLTE3LjgyNC0xMS4zMTZMMTk4LDEzMC41MjF2MTE4LjU3NWMwLDIuNzYxLDIuMjM4LDUsNSw1aDQzICAgYzIuNzYyLDAsNS0yLjIzOSw1LTV2LTg5LjcwNWMtNC41NTMsMy4zODktMTAuMTM5LDUuMjY0LTE1LjgyMSw1LjI2NEMyMzMuODA4LDE2NC42NTQsMjMyLjQyNCwxNjQuNTQ2LDIzMS4wNjQsMTY0LjMzM3oiIGZpbGw9IiMwMDAwMDAiLz4KCTxwYXRoIGQ9Ik0xMjUuMTgxLDE0OS42NTVjMi44NzIsMCw1LjYyMy0xLjA2Niw3Ljc0Ni0zLjAwM2w1OC4xODMtNTMuMDQ4bDM0LjU0OSw1MWMxLjc3OSwyLjYyOCw0LjU5OSw0LjQxOCw3LjczMiw0LjkxICAgYzAuNTkyLDAuMDkzLDEuMTkyLDAuMTQsMS43ODgsMC4xNGMyLjU1MywwLDUuMDY3LTAuODY1LDcuMDc5LTIuNDM2bDg4LjY2OC02OS4yNzhjNC45OTYtMy45MDMsNS44ODYtMTEuMTQ1LDEuOTgyLTE2LjE0MyAgIGMtMi4xOTYtMi44MDktNS41MDEtNC40Mi05LjA2OC00LjQyYy0yLjU1LDAtNS4wNjMsMC44NjYtNy4wNzQsMi40MzhsLTc4Ljk2OSw2MS43bC0zNS4wOTgtNTEuODExICAgYy0xLjg3My0yLjc2Ni00Ljg1LTQuNTc2LTguMTY2LTQuOTdjLTAuNDQ4LTAuMDU0LTAuOTA2LTAuMDgxLTEuMzU3LTAuMDgxYy0yLjg3MiwwLTUuNjIzLDEuMDY2LTcuNzQ2LDMuMDA0bC02Ny45OTksNjEuOTk5ICAgYy0yLjI3LDIuMDY5LTMuNTk5LDQuODk4LTMuNzM5LDcuOTY4Yy0wLjE0MywzLjA2OCwwLjkyLDYuMDA4LDIuOTg5LDguMjc4QzExOC44NTQsMTQ4LjI4NywxMjEuOTUyLDE0OS42NTUsMTI1LjE4MSwxNDkuNjU1eiIgZmlsbD0iIzAwMDAwMCIvPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+Cjwvc3ZnPgo=" />
            </div>
        </div>
    </div>
    
    </br>
    
    <div class="row">
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="time" id="time" onClick="visualizationChange(this)">Time Series</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTYuMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjwhRE9DVFlQRSBzdmcgUFVCTElDICItLy9XM0MvL0RURCBTVkcgMS4xLy9FTiIgImh0dHA6Ly93d3cudzMub3JnL0dyYXBoaWNzL1NWRy8xLjEvRFREL3N2ZzExLmR0ZCI+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4PSIwcHgiIHk9IjBweCIgd2lkdGg9IjMycHgiIGhlaWdodD0iMzJweCIgdmlld0JveD0iMCAwIDEwOC43MjcgMTA4LjcyNyIgc3R5bGU9ImVuYWJsZS1iYWNrZ3JvdW5kOm5ldyAwIDAgMTA4LjcyNyAxMDguNzI3OyIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSI+CjxnPgoJPHBhdGggZD0iTTAsNy45ODh2OTIuNzVoMTA4LjcyN1Y3Ljk4OEgweiBNOTMuMjY0LDM4Ljk4OHYtMTQuNWgxMi4zMjF2LTAuNUg5My4yNjR2LTEzaDEyLjQ2MlY1NC44NCAgIGMtMC4xMzItMC4xMjktMC4yOC0wLjIzNy0wLjQyNi0wLjM1MmgwLjI4NXYtMC41aC0xLjA1OGMtMC43MjgtMC4zODYtMS41NDMtMC42MjUtMi40MjItMC42MjVzLTEuNjkzLDAuMjM5LTIuNDIxLDAuNjI1aC0yLjAxMyAgIGwtNC40MDgtMi43ODlWMzkuNDg4aDEyLjMyMXYtMC41SDkzLjI2NHogTTk4LjQ2Myw1NC40ODhoMC40NDljLTAuMDY1LDAuMDUzLTAuMTMxLDAuMTA1LTAuMTk0LDAuMTYxTDk4LjQ2Myw1NC40ODh6ICAgIE0xMDIuMTA3LDU1LjM2M2MxLjc3LDAsMy4yMDksMS40MzksMy4yMDksMy4yMDlzLTEuNDM5LDMuMjA5LTMuMjA5LDMuMjA5Yy0xLjc3MSwwLTMuMjA5LTEuNDM5LTMuMjA5LTMuMjA5ICAgUzEwMC4zMzcsNTUuMzYzLDEwMi4xMDcsNTUuMzYzeiBNOTMuMjY0LDUzLjU2NWwwLjY2OCwwLjQyM2gtMC42NjhWNTMuNTY1eiBNNzYuNDU5LDQzLjY4OGMwLTEuMDU1LDAuNTE4LTEuOTgzLDEuMzA2LTIuNTY5djUuMTM5ICAgQzc2Ljk3Nyw0NS42NzIsNzYuNDU5LDQ0Ljc0Myw3Ni40NTksNDMuNjg4eiBNNzguMjY0LDQwLjgxNmMwLjQyNi0wLjIwOSwwLjg5OC0wLjMzNywxLjQwMy0wLjMzN2MxLjc3MSwwLDMuMjA5LDEuNDM5LDMuMjA5LDMuMjA5ICAgYzAsMS43Ny0xLjQzOCwzLjIwOS0zLjIwOSwzLjIwOWMtMC41MDUsMC0wLjk3OC0wLjEyOC0xLjQwMy0wLjMzN1Y0MC44MTZ6IE04Mi43MjEsMzkuNDg4aDEwLjA0M3YxMS4zOTVsLTguMjc3LTUuMjM2ICAgYzAuMjQ4LTAuNjA1LDAuMzktMS4yNjUsMC4zOS0xLjk1OEM4NC44NzcsNDEuOTYsODQuMDIxLDQwLjQzNyw4Mi43MjEsMzkuNDg4eiBNNzguMjY0LDEwLjk4OGgxNC41djEzaC0xNC41VjEwLjk4OHogICAgTTkyLjc2NCwyNC40ODh2MTQuNWgtMTAuODljLTAuNjczLTAuMzE3LTEuNDE1LTAuNTA5LTIuMjA3LTAuNTA5Yy0wLjQ4OSwwLTAuOTU0LDAuMDktMS40MDMsMC4yMTdWMjQuNDg4SDkyLjc2NHogTTYzLjI2NCwxMC45ODggICBoMTQuNXYxM2gtMTQuNVYxMC45ODh6IE02My4yNjQsMjQuNDg4aDE0LjV2MTQuMzYzYy0wLjEwNCwwLjA0MS0wLjIwMywwLjA4OS0wLjMwNCwwLjEzN0g2My4yNjRWMjQuNDg4eiBNNjMuMjY0LDM5LjQ4OGgxMy4zNSAgIGMtMS4yOTksMC45NDgtMi4xNTUsMi40NzItMi4xNTUsNC4yYzAsMC45LDAuMjUsMS43MzUsMC42NTMsMi40NzVsLTEwLjkzOCw3LjgyNWgtMC45MVYzOS40ODh6IE02My40NzUsNTQuNDg4bC0wLjIxMSwwLjE1di0wLjE1ICAgSDYzLjQ3NXogTTU5Ljk5Niw1NC40ODhoMi43Njl2MC41MDlsLTEuNDQyLDEuMDMyQzYwLjk4Miw1NS40Myw2MC41MzUsNTQuOTEsNTkuOTk2LDU0LjQ4OHogTTQ4LjI2NCw1MC40MTFsMy41NDMsMy41NzdoLTMuNTQzICAgVjUwLjQxMXogTTU2LjgwMyw1NS4zNjNjMS43NywwLDMuMjA5LDEuNDM5LDMuMjA5LDMuMjA5cy0xLjQzOSwzLjIwOS0zLjIwOSwzLjIwOWMtMS43NywwLTMuMjA5LTEuNDM5LTMuMjA5LTMuMjA5ICAgUzU1LjAzMyw1NS4zNjMsNTYuODAzLDU1LjM2M3ogTTQ4LjI2NCwxMC45ODhoMTQuNXYxM2gtMTQuNVYxMC45ODh6IE00OC4yNjQsMjQuNDg4aDE0LjV2MTQuNWgtMTQuNVYyNC40ODh6IE02Mi43NjQsMzkuNDg4djE0LjUgICBoLTMuNTQxYy0wLjcyNy0wLjM4Ni0xLjU0Mi0wLjYyNS0yLjQyLTAuNjI1Yy0wLjgxNSwwLTEuNTc3LDAuMjA1LTIuMjY0LDAuNTM5bC02LjI3NC02LjMzNHYtOC4wOEg2Mi43NjR6IE00Ny43NjQsNDcuMDY0ICAgbC03LjUwNC03LjU3Nmg3LjUwNFY0Ny4wNjR6IE0zMy4yNjQsMTAuOTg4aDE0LjV2MTNoLTE0LjVWMTAuOTg4eiBNNDcuNzY0LDI0LjQ4OHYxNC41aC04bC0xLjQ0OC0xLjQ2MiAgIGMwLjMzMy0wLjY4NiwwLjUzNS0xLjQ0NCwwLjUzNS0yLjI1NmMwLTIuODcyLTIuMzM3LTUuMjA5LTUuMjA5LTUuMjA5Yy0wLjEyOSwwLTAuMjUxLDAuMDI5LTAuMzc4LDAuMDM4di01LjYxMUg0Ny43NjR6ICAgIE0zMC40MzQsMzUuMjcxYzAtMS40NjMsMC45OS0yLjY4OCwyLjMzMS0zLjA3MXY2LjE0M0MzMS40MjQsMzcuOTU3LDMwLjQzNCwzNi43MzMsMzAuNDM0LDM1LjI3MXogTTMzLjI2NCwzMi4wOTkgICBjMC4xMjUtMC4wMTUsMC4yNDktMC4wMzgsMC4zNzgtMC4wMzhjMS43NywwLDMuMjA5LDEuNDM5LDMuMjA5LDMuMjA5YzAsMS43Ny0xLjQzOSwzLjIwOS0zLjIwOSwzLjIwOSAgIGMtMC4xMjksMC0wLjI1My0wLjAyMy0wLjM3OC0wLjAzOFYzMi4wOTl6IE0xOC4yNjQsMTAuOTg4aDE0LjV2MTNoLTE0LjVWMTAuOTg4eiBNMTguMjY0LDI0LjQ4OGgxNC41djUuNjYyICAgYy0yLjQ1MywwLjQyLTQuMzMxLDIuNTQ5LTQuMzMxLDUuMTJjMCwwLjg1OCwwLjIyOSwxLjY1NiwwLjU5NywyLjM3bC0wLjkzMSwxLjM0OGgtOS44MzZMMTguMjY0LDI0LjQ4OEwxOC4yNjQsMjQuNDg4eiAgICBNMjcuNzU1LDM5LjQ4OGwtOS40OSwxMy43MzZWMzkuNDg4SDI3Ljc1NXogTTE3Ljc2NCwxMC45ODh2MTNIMy4xNHYwLjVoMTQuNjI0djE0LjVIMy4xNHYwLjVoMTQuNjI0djE0LjQ2bC0wLjAyOCwwLjA0SDMuMTR2MC41ICAgaDE0LjI1MWwtMi4yMzUsMy4yMzRjLTAuOTA2LTAuNzY1LTIuMDYxLTEuMjQyLTMuMzM1LTEuMjQyYy0yLjg3MiwwLTUuMjA5LDIuMzM3LTUuMjA5LDUuMjA5YzAsMS40MjYsMC41NzgsMi43MTksMS41MSwzLjY2ICAgbC0zLjU0NCwzLjYzOUgzLjE0djAuNWgwLjk1MUwzLDcwLjYwOHYtNTkuNjJIMTcuNzY0eiBNOC42MTIsNjEuNjg5YzAtMS43NywxLjQzOS0zLjIwOSwzLjIwOS0zLjIwOXMzLjIwOSwxLjQzOSwzLjIwOSwzLjIwOSAgIHMtMS40MzksMy4yMDktMy4yMDksMy4yMDlTOC42MTIsNjMuNDU5LDguNjEyLDYxLjY4OXogTTE3Ljc2NCw4My45ODhIMy4xNHYwLjVoMTQuNjI0djEzLjI1SDNWNzMuNDczbDMuODgyLTMuOTg0aDEwLjg4MlY4My45ODh6ICAgIE03LjM2OSw2OC45ODhsMi40MzMtMi40OThjMC42MjEsMC4yNjQsMS4zMDMsMC40MDgsMi4wMTksMC40MDhjMi44NzIsMCw1LjIwOS0yLjMzNyw1LjIwOS01LjIwOWMwLTAuODQyLTAuMjE5LTEuNjI1LTAuNTc1LTIuMzI4ICAgbDEuMzA5LTEuODk1djExLjUyMUw3LjM2OSw2OC45ODhMNy4zNjksNjguOTg4eiBNMzIuNzY0LDk3LjczOGgtMTQuNXYtMTMuMjVoMTQuNVY5Ny43Mzh6IE0zMi43NjQsODMuOTg4aC0xNC41di0xNC41aDE0LjVWODMuOTg4ICAgeiBNMzIuNzY0LDY4Ljk4OGgtMTQuNVY1Ni43NDNsMS41NTgtMi4yNTVoMTIuOTQyVjY4Ljk4OHogTTIwLjE2OCw1My45ODhsMTAuMDE4LTE0LjVoMC40MjVjMC42MjksMC40NTMsMS4zNTksMC43NjYsMi4xNTMsMC45MDIgICB2MTMuNTk4SDIwLjE2OHogTTQ3Ljc2NCw5Ny43MzhoLTE0LjV2LTEzLjI1aDE0LjVWOTcuNzM4eiBNNDcuNzY0LDgzLjk4OGgtMTQuNXYtMTQuNWgxNC41VjgzLjk4OHogTTQ3Ljc2NCw2OC45ODhoLTE0LjV2LTE0LjUgICBoMTQuNVY2OC45ODh6IE0zMy4yNjQsNTMuOTg4VjQwLjQ0MWMwLjEyNywwLjAwOSwwLjI0OSwwLjAzOCwwLjM3OCwwLjAzOGMxLjEzNCwwLDIuMTc2LTAuMzc0LDMuMDMyLTAuOTkxaDAuNzdsMTAuMzIsMTAuNDE4ICAgdjQuMDgySDMzLjI2NHogTTYyLjc2NCw5Ny43MzhoLTE0LjV2LTEzLjI1aDE0LjVWOTcuNzM4eiBNNjIuNzY0LDgzLjk4OGgtMTQuNXYtMTQuNWgxNC41VjgzLjk4OHogTTQ4LjI2NCw2OC45ODh2LTE0LjVoNC4wMzkgICBsMC42MjgsMC42MzRjLTAuODIyLDAuOTIxLTEuMzM3LDIuMTIyLTEuMzM3LDMuNDVjMCwyLjg3MiwyLjMzNyw1LjIwOSw1LjIwOSw1LjIwOXM1LjIwOS0yLjMzNyw1LjIwOS01LjIwOSAgIGMwLTAuMTg2LTAuMDM2LTAuMzU5LTAuMDU0LTAuNTM4bDAuODA3LTAuNTc3djExLjUzMUg0OC4yNjR6IE03Ny43NjQsOTcuNzM4aC0xNC41di0xMy4yNWgxNC41Vjk3LjczOHogTTc3Ljc2NCw4My45ODhoLTE0LjUgICB2LTE0LjVoMTQuNVY4My45ODh6IE03Ny43NjQsNjguOTg4aC0xNC41VjU3LjA5OGwzLjY0OC0yLjYwOWgxMC44NTJWNjguOTg4eiBNNjcuNjEzLDUzLjk4OGw4Ljc4LTYuMjgxICAgYzAuNDEzLDAuMzM3LDAuODY3LDAuNjE5LDEuMzcsMC44MTh2NS40NjNINjcuNjEzeiBNOTIuNzY0LDk3LjczOGgtMTQuNXYtMTMuMjVoMTQuNVY5Ny43Mzh6IE05Mi43NjQsODMuOTg4aC0xNC41di0xNC41aDE0LjUgICBWODMuOTg4eiBNOTIuNzY0LDY4Ljk4OGgtMTQuNXYtMTQuNWgxNC41VjY4Ljk4OHogTTc4LjI2NCw1My45ODhWNDguNjhjMC40NDksMC4xMjcsMC45MTQsMC4yMTcsMS40MDMsMC4yMTcgICBjMS40NiwwLDIuNzc2LTAuNjA3LDMuNzI0LTEuNTc5bDkuMzczLDUuOTMxdjAuNzM5SDc4LjI2NHogTTkzLjI2NCw5Ny43Mzh2LTEzLjI1aDEyLjMyMXYtMC41SDkzLjI2NHYtMTQuNWgxMi4zMjF2LTAuNUg5My4yNjQgICB2LTE0LjVoMS40NTlsMi43NTYsMS43NDNjLTAuMzU5LDAuNzA3LTAuNTgyLDEuNDk1LTAuNTgyLDIuMzQxYzAsMi44NzIsMi4zMzgsNS4yMDksNS4yMDksNS4yMDljMS40MDcsMCwyLjY4Mi0wLjU2NiwzLjYyLTEuNDc3ICAgdjM1LjQzNEg5My4yNjR6IiBmaWxsPSIjMDAwMDAwIi8+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPC9zdmc+Cg==" />
            </div>
        </div>
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="heatmap" id="heatmap" onClick="visualizationChange(this)">Heatmap</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTYuMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjwhRE9DVFlQRSBzdmcgUFVCTElDICItLy9XM0MvL0RURCBTVkcgMS4xLy9FTiIgImh0dHA6Ly93d3cudzMub3JnL0dyYXBoaWNzL1NWRy8xLjEvRFREL3N2ZzExLmR0ZCI+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4PSIwcHgiIHk9IjBweCIgd2lkdGg9IjMycHgiIGhlaWdodD0iMzJweCIgdmlld0JveD0iMCAwIDM1IDM1IiBzdHlsZT0iZW5hYmxlLWJhY2tncm91bmQ6bmV3IDAgMCAzNSAzNTsiIHhtbDpzcGFjZT0icHJlc2VydmUiPgo8Zz4KCTxnPgoJCTxyZWN0IHk9IjI1LjgxOCIgd2lkdGg9IjE1LjUiIGhlaWdodD0iOS4xODIiIGZpbGw9IiMwMDAwMDAiLz4KCQk8cmVjdCB4PSIxOS41IiB5PSIyNS44MTgiIHdpZHRoPSIxNS41IiBoZWlnaHQ9IjkuMTgyIiBmaWxsPSIjMDAwMDAwIi8+CgkJPHJlY3QgeD0iMTkuNSIgeT0iMTIuODk4IiB3aWR0aD0iMTUuNSIgaGVpZ2h0PSI5LjIwMiIgZmlsbD0iIzAwMDAwMCIvPgoJCTxyZWN0IHk9IjEyLjg5OCIgd2lkdGg9IjE1LjUiIGhlaWdodD0iOS4yMDIiIGZpbGw9IiMwMDAwMDAiLz4KCQk8cmVjdCB3aWR0aD0iMTUuNSIgaGVpZ2h0PSI5LjE4IiBmaWxsPSIjMDAwMDAwIi8+CgkJPHJlY3QgeD0iMTkuNSIgd2lkdGg9IjE1LjUiIGhlaWdodD0iOS4xOCIgZmlsbD0iIzAwMDAwMCIvPgoJPC9nPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+Cjwvc3ZnPgo=" />
            </div>
        </div>
    </div>
    
    </br>
    
    <div class="row">
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="bubble" id="bubble" onClick="visualizationChange(this)">Bubble Chart</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTkuMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4PSIwcHgiIHk9IjBweCIgdmlld0JveD0iMCAwIDQ4MCA0ODAiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDQ4MCA0ODA7IiB4bWw6c3BhY2U9InByZXNlcnZlIiB3aWR0aD0iMzJweCIgaGVpZ2h0PSIzMnB4Ij4KPGc+Cgk8Zz4KCQk8cGF0aCBkPSJNMTA0LDBDNDYuNTYyLDAsMCw0Ni41NjIsMCwxMDRjMC4wNjYsNTcuNDEsNDYuNTksMTAzLjkzNCwxMDQsMTA0YzU3LjQzOCwwLDEwNC00Ni41NjIsMTA0LTEwNFMxNjEuNDM4LDAsMTA0LDB6ICAgICBNMTA0LDE5MmMtNDguNjAxLDAtODgtMzkuMzk5LTg4LTg4YzAuMDU3LTQ4LjU3NywzOS40MjMtODcuOTQzLDg4LTg4YzQ4LjYwMSwwLDg4LDM5LjM5OSw4OCw4OFMxNTIuNjAxLDE5MiwxMDQsMTkyeiIgZmlsbD0iIzAwMDAwMCIvPgoJPC9nPgo8L2c+CjxnPgoJPGc+CgkJPHBhdGggZD0iTTE2MCwxMDRjLTAuMDM1LDMwLjkxMy0yNS4wODcsNTUuOTY1LTU2LDU2djE2YzM5Ljc0Ni0wLjA0NCw3MS45NTYtMzIuMjU0LDcyLTcySDE2MHoiIGZpbGw9IiMwMDAwMDAiLz4KCTwvZz4KPC9nPgo8Zz4KCTxnPgoJCTxwYXRoIGQ9Ik0zOTIsMzJjLTQ4LjYwMSwwLTg4LDM5LjM5OS04OCw4OHMzOS4zOTksODgsODgsODhjNDguNjAxLDAsODgtMzkuMzk5LDg4LTg4QzQ3OS45NDMsNzEuNDIzLDQ0MC41NzcsMzIuMDU3LDM5MiwzMnogICAgIE0zOTIsMTkyYy0zOS43NjQsMC03Mi0zMi4yMzYtNzItNzJjMC0zOS43NjUsMzIuMjM2LTcyLDcyLTcyYzM5Ljc2NSwwLDcyLDMyLjIzNSw3Miw3MkM0NjMuOTU2LDE1OS43NDYsNDMxLjc0NiwxOTEuOTU2LDM5MiwxOTIgICAgeiIgZmlsbD0iIzAwMDAwMCIvPgoJPC9nPgo8L2c+CjxnPgoJPGc+CgkJPHBhdGggZD0iTTMzNiwxMjBoMTZjMC4wMjYtMjIuMDgsMTcuOTItMzkuOTc0LDQwLTQwVjY0QzM2MS4wODcsNjQuMDM1LDMzNi4wMzUsODkuMDg3LDMzNiwxMjB6IiBmaWxsPSIjMDAwMDAwIi8+Cgk8L2c+CjwvZz4KPGc+Cgk8Zz4KCQk8cGF0aCBkPSJNMTc2LDMwNGMtNDguNjAxLDAtODgsMzkuMzk5LTg4LDg4YzAsNDguNjAxLDM5LjM5OSw4OCw4OCw4OHM4OC0zOS4zOTksODgtODhDMjYzLjk0MywzNDMuNDIzLDIyNC41NzcsMzA0LjA1NywxNzYsMzA0eiAgICAgTTE3Niw0NjRjLTM5Ljc2NCwwLTcyLTMyLjIzNi03Mi03MmMwLTM5Ljc2NSwzMi4yMzYtNzIsNzItNzJjMzkuNzY1LDAsNzIsMzIuMjM1LDcyLDcyQzI0Ny45NTYsNDMxLjc0NiwyMTUuNzQ2LDQ2My45NTYsMTc2LDQ2NCAgICB6IiBmaWxsPSIjMDAwMDAwIi8+Cgk8L2c+CjwvZz4KPGc+Cgk8Zz4KCQk8cGF0aCBkPSJNMTM2LDM5MmgtMTZjMC4wMzUsMzAuOTEzLDI1LjA4Nyw1NS45NjUsNTYsNTZ2LTE2QzE1My45Miw0MzEuOTc0LDEzNi4wMjYsNDE0LjA4LDEzNiwzOTJ6IiBmaWxsPSIjMDAwMDAwIi8+Cgk8L2c+CjwvZz4KPGc+Cgk8Zz4KCQk8cGF0aCBkPSJNMjQwLDE5MmMtMjYuNTEsMC00OCwyMS40OS00OCw0OGMwLDI2LjUxLDIxLjQ5LDQ4LDQ4LDQ4YzI2LjUxLDAsNDgtMjEuNDksNDgtNDggICAgQzI4Ny45NzQsMjEzLjUwMSwyNjYuNDk5LDE5Mi4wMjYsMjQwLDE5MnogTTI0MCwyNzJjLTE3LjY3MywwLTMyLTE0LjMyNy0zMi0zMnMxNC4zMjctMzIsMzItMzJjMTcuNjczLDAsMzIsMTQuMzI3LDMyLDMyICAgIFMyNTcuNjczLDI3MiwyNDAsMjcyeiIgZmlsbD0iIzAwMDAwMCIvPgoJPC9nPgo8L2c+CjxnPgoJPGc+CgkJPHBhdGggZD0iTTQwMCwyNzJjLTI2LjUxLDAtNDgsMjEuNDktNDgsNDhjMCwyNi41MSwyMS40OSw0OCw0OCw0OGMyNi41MSwwLDQ4LTIxLjQ5LDQ4LTQ4ICAgIEM0NDcuOTc0LDI5My41MDEsNDI2LjQ5OSwyNzIuMDI2LDQwMCwyNzJ6IE00MDAsMzUyYy0xNy42NzMsMC0zMi0xNC4zMjctMzItMzJjMC0xNy42NzMsMTQuMzI3LTMyLDMyLTMyICAgIGMxNy42NzMsMCwzMiwxNC4zMjcsMzIsMzJDNDMyLDMzNy42NzMsNDE3LjY3MywzNTIsNDAwLDM1MnoiIGZpbGw9IiMwMDAwMDAiLz4KCTwvZz4KPC9nPgo8Zz4KCTxnPgoJCTxwYXRoIGQ9Ik00OCwyNDhjLTEzLjI1NSwwLTI0LDEwLjc0NS0yNCwyNHMxMC43NDUsMjQsMjQsMjRzMjQtMTAuNzQ1LDI0LTI0UzYxLjI1NSwyNDgsNDgsMjQ4eiBNNDgsMjgwYy00LjQxOCwwLTgtMy41ODItOC04ICAgIHMzLjU4Mi04LDgtOHM4LDMuNTgyLDgsOFM1Mi40MTgsMjgwLDQ4LDI4MHoiIGZpbGw9IiMwMDAwMDAiLz4KCTwvZz4KPC9nPgo8Zz4KCTxnPgoJCTxwYXRoIGQ9Ik0yNDAsMjRjLTEzLjI1NSwwLTI0LDEwLjc0NS0yNCwyNHMxMC43NDUsMjQsMjQsMjRzMjQtMTAuNzQ1LDI0LTI0UzI1My4yNTUsMjQsMjQwLDI0eiBNMjQwLDU2Yy00LjQxOCwwLTgtMy41ODItOC04ICAgIHMzLjU4Mi04LDgtOHM4LDMuNTgyLDgsOFMyNDQuNDE4LDU2LDI0MCw1NnoiIGZpbGw9IiMwMDAwMDAiLz4KCTwvZz4KPC9nPgo8Zz4KCTxnPgoJCTxwYXRoIGQ9Ik0zNDQsNDA4Yy0xMy4yNTUsMC0yNCwxMC43NDUtMjQsMjRzMTAuNzQ1LDI0LDI0LDI0czI0LTEwLjc0NSwyNC0yNFMzNTcuMjU1LDQwOCwzNDQsNDA4eiBNMzQ0LDQ0MGMtNC40MTgsMC04LTMuNTgyLTgtOCAgICBzMy41ODItOCw4LThzOCwzLjU4Miw4LDhTMzQ4LjQxOCw0NDAsMzQ0LDQ0MHoiIGZpbGw9IiMwMDAwMDAiLz4KCTwvZz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8L3N2Zz4K" />
            </div>
        </div>
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="map" id="map" onClick="visualizationChange(this)">Map</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTYuMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjwhRE9DVFlQRSBzdmcgUFVCTElDICItLy9XM0MvL0RURCBTVkcgMS4xLy9FTiIgImh0dHA6Ly93d3cudzMub3JnL0dyYXBoaWNzL1NWRy8xLjEvRFREL3N2ZzExLmR0ZCI+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4PSIwcHgiIHk9IjBweCIgd2lkdGg9IjMycHgiIGhlaWdodD0iMzJweCIgdmlld0JveD0iMCAwIDUxMiA1MTIiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDUxMiA1MTI7IiB4bWw6c3BhY2U9InByZXNlcnZlIj4KPGc+Cgk8cGF0aCBkPSJNNDE2LDQ4Yy00NC4xODgsMC04MCwzNS44MTMtODAsODBjMCwxMS45MzgsMi42MjUsMjMuMjgxLDcuMzEyLDMzLjQzOEw0MTYsMzA0bDcyLjY4OC0xNDIuNTYzICAgQzQ5My4zNzUsMTUxLjI4MSw0OTYsMTM5LjkzOCw0OTYsMTI4QzQ5Niw4My44MTMsNDYwLjE4OCw0OCw0MTYsNDh6IE00MTYsMTc2Yy0yNi41LDAtNDgtMjEuNS00OC00OHMyMS41LTQ4LDQ4LTQ4czQ4LDIxLjUsNDgsNDggICBTNDQyLjUsMTc2LDQxNiwxNzZ6IE00MzkuOTM4LDMyNy40NjlsMjkuMTI1LDU4LjIxOWwtNzMuODQ0LDM2LjkwNmwtMjQuNzUtMTIzLjgxMmw0LjE1Ni00LjE1NmwwLjQzOC0wLjQzOGwtMTUuMjUtMzBMMzUyLDI3MiAgIGwtOTYtNjRsLTk2LDY0bC02NC02NEwwLDQwMGwxMjgsNjRsMTI4LTY0bDEyOCw2NGwxMjgtNjRsLTU0LTEwNy45NjlMNDM5LjkzOCwzMjcuNDY5eiBNMTE2Ljc1LDQyMi41OTRsLTczLjgxMy0zNi45MDZMMTA0Ljc1LDI2MiAgIGwzMi42MjUsMzIuNjI1bDQuMTU2LDQuMTU2TDExNi43NSw0MjIuNTk0eiBNMjQwLDM3Mi4yMTlsLTg5LjUsNDQuNzVsMjMuMTI1LTExNS41OTRsNC4xMjUtMi43NWw2Mi4yNS00MS41VjM3Mi4yMTl6IE0yNzIsMzcyLjIxOSAgIFYyNTcuMTI1bDYyLjI1LDQxLjVsNC4wOTQsMi43NWwyMy4xMjUsMTE1LjU5NEwyNzIsMzcyLjIxOXoiIGZpbGw9IiMwMDAwMDAiLz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8Zz4KPC9nPgo8L3N2Zz4K" />
            </div>
        </div>
    </div>
    
    </br>
    
    <div class="row">
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="grid" id="grid" onClick="visualizationChange(this)">Grid View</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTYuMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjwhRE9DVFlQRSBzdmcgUFVCTElDICItLy9XM0MvL0RURCBTVkcgMS4xLy9FTiIgImh0dHA6Ly93d3cudzMub3JnL0dyYXBoaWNzL1NWRy8xLjEvRFREL3N2ZzExLmR0ZCI+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4PSIwcHgiIHk9IjBweCIgd2lkdGg9IjMycHgiIGhlaWdodD0iMzJweCIgdmlld0JveD0iMCAwIDQ3NS4wODIgNDc1LjA4MSIgc3R5bGU9ImVuYWJsZS1iYWNrZ3JvdW5kOm5ldyAwIDAgNDc1LjA4MiA0NzUuMDgxOyIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSI+CjxnPgoJPHBhdGggZD0iTTQ2MS42NjcsNDkuOTYzYy04Ljk0OS04Ljk0Ny0xOS42OTgtMTMuNDE4LTMyLjI2NS0xMy40MThINDUuNjgyYy0xMi41NjIsMC0yMy4zMTcsNC40NzEtMzIuMjY0LDEzLjQxOCAgIEM0LjQ3Myw1OC45MTIsMCw2OS42NjMsMCw4Mi4yMjhWMzkyLjg2YzAsMTIuNTY2LDQuNDczLDIzLjMwOSwxMy40MTgsMzIuMjYxYzguOTQ3LDguOTQ5LDE5LjcwMSwxMy40MTUsMzIuMjY0LDEzLjQxNWgzODMuNzIgICBjMTIuNTY2LDAsMjMuMzE1LTQuNDY2LDMyLjI2NS0xMy40MTVjOC45NDUtOC45NTIsMTMuNDE1LTE5LjcwMSwxMy40MTUtMzIuMjYxVjgyLjIyOCAgIEM0NzUuMDgyLDY5LjY2Myw0NzAuNjEyLDU4LjkwOSw0NjEuNjY3LDQ5Ljk2M3ogTTE0Ni4xODMsMzkyLjg1YzAsMi42NzMtMC44NTksNC44NTYtMi41NzQsNi41NzEgICBjLTEuNzEyLDEuNzExLTMuODk5LDIuNTYyLTYuNTY3LDIuNTYyaC05MS4zNmMtMi42NjIsMC00Ljg1My0wLjg1Mi02LjU2Ny0yLjU2MmMtMS43MTMtMS43MTUtMi41NjgtMy44OTgtMi41NjgtNi41NzFWMzM4LjAzICAgYzAtMi42NjksMC44NTUtNC44NTMsMi41NjgtNi41NmMxLjcxNC0xLjcxOSwzLjkwNS0yLjU3NCw2LjU2Ny0yLjU3NGg5MS4zNjNjMi42NjcsMCw0Ljg1OCwwLjg1NSw2LjU2NywyLjU3NCAgIGMxLjcxMSwxLjcwNywyLjU3LDMuODkxLDIuNTcsNi41NlYzOTIuODV6IE0xNDYuMTgzLDI4My4yMjFjMCwyLjY2My0wLjg1OSw0Ljg1NC0yLjU3NCw2LjU2NCAgIGMtMS43MTIsMS43MTQtMy44OTksMi41NjktNi41NjcsMi41NjloLTkxLjM2Yy0yLjY2MiwwLTQuODUzLTAuODU1LTYuNTY3LTIuNTY5Yy0xLjcxMy0xLjcxMS0yLjU2OC0zLjkwMS0yLjU2OC02LjU2NHYtNTQuODE5ICAgYzAtMi42NjQsMC44NTUtNC44NTQsMi41NjgtNi41NjdjMS43MTQtMS43MDksMy45MDUtMi41NjUsNi41NjctMi41NjVoOTEuMzYzYzIuNjY3LDAsNC44NTQsMC44NTUsNi41NjcsMi41NjUgICBjMS43MTEsMS43MTMsMi41NywzLjkwMywyLjU3LDYuNTY3VjI4My4yMjF6IE0xNDYuMTgzLDE3My41ODdjMCwyLjY2Ni0wLjg1OSw0Ljg1My0yLjU3NCw2LjU2NyAgIGMtMS43MTIsMS43MDktMy44OTksMi41NjgtNi41NjcsMi41NjhoLTkxLjM2Yy0yLjY2MiwwLTQuODUzLTAuODU5LTYuNTY3LTIuNTY4Yy0xLjcxMy0xLjcxNS0yLjU2OC0zLjkwMS0yLjU2OC02LjU2N1YxMTguNzcgICBjMC0yLjY2NiwwLjg1NS00Ljg1NiwyLjU2OC02LjU2N2MxLjcxNC0xLjcxMywzLjkwNS0yLjU2OCw2LjU2Ny0yLjU2OGg5MS4zNjNjMi42NjcsMCw0Ljg1NCwwLjg1NSw2LjU2NywyLjU2OCAgIGMxLjcxMSwxLjcxMSwyLjU3LDMuOTAxLDIuNTcsNi41NjdWMTczLjU4N3ogTTI5Mi4zNjIsMzkyLjg1YzAsMi42NzMtMC44NTUsNC44NTYtMi41NjMsNi41NzFjLTEuNzExLDEuNzExLTMuOSwyLjU2Mi02LjU3LDIuNTYyICAgSDE5MS44NmMtMi42NjMsMC00Ljg1My0wLjg1Mi02LjU2Ny0yLjU2MmMtMS43MTMtMS43MTUtMi41NjgtMy44OTgtMi41NjgtNi41NzFWMzM4LjAzYzAtMi42NjksMC44NTUtNC44NTMsMi41NjgtNi41NiAgIGMxLjcxNC0xLjcxOSwzLjkwNC0yLjU3NCw2LjU2Ny0yLjU3NGg5MS4zNjVjMi42NjksMCw0Ljg1OSwwLjg1NSw2LjU3LDIuNTc0YzEuNzA0LDEuNzA3LDIuNTYsMy44OTEsMi41Niw2LjU2djU0LjgxOUgyOTIuMzYyeiAgICBNMjkyLjM2MiwyODMuMjIxYzAsMi42NjMtMC44NTUsNC44NTQtMi41NjMsNi41NjRjLTEuNzExLDEuNzE0LTMuOSwyLjU2OS02LjU3LDIuNTY5SDE5MS44NmMtMi42NjMsMC00Ljg1My0wLjg1NS02LjU2Ny0yLjU2OSAgIGMtMS43MTMtMS43MTEtMi41NjgtMy45MDEtMi41NjgtNi41NjR2LTU0LjgxOWMwLTIuNjY0LDAuODU1LTQuODU0LDIuNTY4LTYuNTY3YzEuNzE0LTEuNzA5LDMuOTA0LTIuNTY1LDYuNTY3LTIuNTY1aDkxLjM2NSAgIGMyLjY2OSwwLDQuODU5LDAuODU1LDYuNTcsMi41NjVjMS43MDQsMS43MTMsMi41NiwzLjkwMywyLjU2LDYuNTY3djU0LjgxOUgyOTIuMzYyeiBNMjkyLjM2MiwxNzMuNTg3ICAgYzAsMi42NjYtMC44NTUsNC44NTMtMi41NjMsNi41NjdjLTEuNzExLDEuNzA5LTMuOSwyLjU2OC02LjU3LDIuNTY4SDE5MS44NmMtMi42NjMsMC00Ljg1My0wLjg1OS02LjU2Ny0yLjU2OCAgIGMtMS43MTMtMS43MTUtMi41NjgtMy45MDEtMi41NjgtNi41NjdWMTE4Ljc3YzAtMi42NjYsMC44NTUtNC44NTYsMi41NjgtNi41NjdjMS43MTQtMS43MTMsMy45MDQtMi41NjgsNi41NjctMi41NjhoOTEuMzY1ICAgYzIuNjY5LDAsNC44NTksMC44NTUsNi41NywyLjU2OGMxLjcwNCwxLjcxMSwyLjU2LDMuOTAxLDIuNTYsNi41Njd2NTQuODE3SDI5Mi4zNjJ6IE00MzguNTM2LDM5Mi44NSAgIGMwLDIuNjczLTAuODU1LDQuODU2LTIuNTYyLDYuNTcxYy0xLjcxOCwxLjcxMS0zLjkwOCwyLjU2Mi02LjU3MSwyLjU2MmgtOTEuMzU0Yy0yLjY3MywwLTQuODYyLTAuODUyLTYuNTctMi41NjIgICBjLTEuNzExLTEuNzE1LTIuNTYtMy44OTgtMi41Ni02LjU3MVYzMzguMDNjMC0yLjY2OSwwLjg0OS00Ljg1MywyLjU2LTYuNTZjMS43MDgtMS43MTksMy44OTctMi41NzQsNi41Ny0yLjU3NGg5MS4zNTQgICBjMi42NjMsMCw0Ljg1NCwwLjg1NSw2LjU3MSwyLjU3NGMxLjcwNywxLjcwNywyLjU2MiwzLjg5MSwyLjU2Miw2LjU2VjM5Mi44NXogTTQzOC41MzYsMjgzLjIyMWMwLDIuNjYzLTAuODU1LDQuODU0LTIuNTYyLDYuNTY0ICAgYy0xLjcxOCwxLjcxNC0zLjkwOCwyLjU2OS02LjU3MSwyLjU2OWgtOTEuMzU0Yy0yLjY3MywwLTQuODYyLTAuODU1LTYuNTctMi41NjljLTEuNzExLTEuNzExLTIuNTYtMy45MDEtMi41Ni02LjU2NHYtNTQuODE5ICAgYzAtMi42NjQsMC44NDktNC44NTQsMi41Ni02LjU2N2MxLjcwOC0xLjcwOSwzLjg5Ny0yLjU2NSw2LjU3LTIuNTY1aDkxLjM1NGMyLjY2MywwLDQuODU0LDAuODU1LDYuNTcxLDIuNTY1ICAgYzEuNzA3LDEuNzEzLDIuNTYyLDMuOTAzLDIuNTYyLDYuNTY3VjI4My4yMjF6IE00MzguNTM2LDE3My41ODdjMCwyLjY2Ni0wLjg1NSw0Ljg1My0yLjU2Miw2LjU2NyAgIGMtMS43MTgsMS43MDktMy45MDgsMi41NjgtNi41NzEsMi41NjhoLTkxLjM1NGMtMi42NzMsMC00Ljg2Mi0wLjg1OS02LjU3LTIuNTY4Yy0xLjcxMS0xLjcxNS0yLjU2LTMuOTAxLTIuNTYtNi41NjdWMTE4Ljc3ICAgYzAtMi42NjYsMC44NDktNC44NTYsMi41Ni02LjU2N2MxLjcwOC0xLjcxMywzLjg5Ny0yLjU2OCw2LjU3LTIuNTY4aDkxLjM1NGMyLjY2MywwLDQuODU0LDAuODU1LDYuNTcxLDIuNTY4ICAgYzEuNzA3LDEuNzExLDIuNTYyLDMuOTAxLDIuNTYyLDYuNTY3VjE3My41ODd6IiBmaWxsPSIjMDAwMDAwIi8+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPC9zdmc+Cg==" />
            </div>
        </div>
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="area" id="area" onClick="visualizationChange(this)">Area Chart</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIj8+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiBoZWlnaHQ9IjMycHgiIHZpZXdCb3g9IjAgMCA0ODAgNDgwIiB3aWR0aD0iMzJweCI+PHBhdGggZD0ibTQ3MiAxNTUuMzEyNS02Mi4zNDM3NSA2Mi4zNDM3NWMtMy4xMjUgMy4xMjEwOTQtOC4xODc1IDMuMTIxMDk0LTExLjMxMjUgMGwtMzQuMzQzNzUtMzQuMzQzNzUtOTAuMzQzNzUgOTAuMzQzNzVjLTMuMTI1IDMuMTIxMDk0LTguMTg3NSAzLjEyMTA5NC0xMS4zMTI1IDBsLTIyLjM0Mzc1LTIyLjM0Mzc1LTgyLjM0Mzc1IDgyLjM0Mzc1Yy0zLjEyNSAzLjEyMTA5NC04LjE4NzUgMy4xMjEwOTQtMTEuMzEyNSAwbC0yNi4zNDM3NS0yNi4zNDM3NS00MCA0MHY1Mi42ODc1aDM5MnptMCAwIiBmaWxsPSIjMDAwMDAwIi8+PHBhdGggZD0ibTU2IDQyNHYtNDI0aC0xNnY0MGgtMjR2MTZoMjR2MTZoLTI0djE2aDI0djI0aC0yNHYxNmgyNHYyNGgtMjR2MTZoMjR2MjRoLTI0djE2aDI0djI0aC0yNHYxNmgyNHYyNGgtMjR2MTZoMjR2MjRoLTI0djE2aDI0djI0aC0yNHYxNmgyNHYyNGgtMjR2MTZoMjR2MTZoLTQwdjE2aDQwdjQwaDE2di00MGgyNHYzMmgxNnYtMzJoMjR2MzJoMTZ2LTMyaDI0djMyaDE2di0zMmgyNHYzMmgxNnYtMzJoMjR2MzJoMTZ2LTMyaDI0djMyaDE2di0zMmgyNHYzMmgxNnYtMzJoMjR2MzJoMTZ2LTMyaDI0djMyaDE2di0zMmgyNHYzMmgxNnYtMzJoMjR2LTE2em0wIDAiIGZpbGw9IiMwMDAwMDAiLz48L3N2Zz4K" />
            </div>
        </div>        
    </div>
    
    </br>
    
    <div class="row">
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="spline" id="spline" onClick="visualizationChange(this)">Spline</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTkuMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iTGF5ZXJfMSIgeD0iMHB4IiB5PSIwcHgiIHZpZXdCb3g9IjAgMCA1MTIgNTEyIiBzdHlsZT0iZW5hYmxlLWJhY2tncm91bmQ6bmV3IDAgMCA1MTIgNTEyOyIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSIgd2lkdGg9IjMycHgiIGhlaWdodD0iMzJweCI+CjxnPgoJPGc+CgkJPHBhdGggZD0iTTUwMy40NjcsMGgtNTEuMmMtNC43MSwwLTguNTMzLDMuODE0LTguNTMzLDguNTMzdjUxLjJjMCw0LjcxOSwzLjgyMyw4LjUzMyw4LjUzMyw4LjUzM2gxNi4wNzcgICAgYy0xNS4wMjcsMTM2LjEzMi0zMS4wOTUsMjQzLjM1NC03NS44MSwyNzUuNjc4VjMzMi44YzAtNC43MTktMy44MjMtOC41MzMtOC41MzMtOC41MzNoLTUxLjJjLTQuNzEsMC04LjUzMywzLjgxNC04LjUzMyw4LjUzMyAgICB2OS4yMDdjLTI0LjIyNi0yMC41OTEtNDcuNTktNjAuNDUtNzAuMjk4LTk5LjI2Yy0yMi40ODUtMzguNDI2LTQzLjc5My03NC43OTUtNjYuMjM2LTkzLjI2OVYxMjhjMC00LjcxOS0zLjgyMy04LjUzMy04LjUzMy04LjUzMyAgICBIMTI4Yy00LjcxLDAtOC41MzMsMy44MTQtOC41MzMsOC41MzN2MjAuMzI2Yy00NS44MzMsMjUuMjA3LTczLjkxNiwxMTQuNjk3LTkzLjAwNSwyOTUuNDA3SDguNTMzICAgIGMtNC43MSwwLTguNTMzLDMuODE0LTguNTMzLDguNTMzdjUxLjJDMCw1MDguMTg2LDMuODIzLDUxMiw4LjUzMyw1MTJoNTEuMmM0LjcxLDAsOC41MzMtMy44MTQsOC41MzMtOC41MzN2LTUxLjIgICAgYzAtNC43MTktMy44MjMtOC41MzMtOC41MzMtOC41MzNINDMuNjIyYzE2LjczNC0xNTcuMTI0LDQxLjA1NC0yNDUuNTk4LDc1Ljg0NC0yNzQuNzY1VjE3OS4yYzAsNC43MTksMy44MjMsOC41MzMsOC41MzMsOC41MzMgICAgaDUxLjJjNC43MSwwLDguNTMzLTMuODE0LDguNTMzLTguNTMzdi01Ljk3M2MxNi42MTQsMTguNTYsMzMuNjY0LDQ3LjY2Nyw1MS40OTksNzguMTQgICAgYzI2LjUzOSw0NS4zNjMsNTMuOTQ4LDkyLjExNyw4NS4wMzUsMTExLjgyOVYzODRjMCw0LjcxOSwzLjgyMyw4LjUzMyw4LjUzMyw4LjUzM0gzODRjNC43MSwwLDguNTMzLTMuODE0LDguNTMzLTguNTMzdi0yMC4wOTYgICAgYzU4LjUzOS0yOS4xNTgsNzUuOTgxLTE0MS4yMSw5Mi45NzktMjk1LjYzN2gxNy45NTRjNC43MSwwLDguNTMzLTMuODE0LDguNTMzLTguNTMzdi01MS4yQzUxMiwzLjgxNCw1MDguMTc3LDAsNTAzLjQ2NywweiIgZmlsbD0iIzAwMDAwMCIvPgoJPC9nPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+CjxnPgo8L2c+Cjwvc3ZnPgo=" />
            </div>
        </div>
        <div class="col col-md-6">
            <div class="radio">
                <label class="col col-md-4" style="margin-top:5px;"><input type="radio" name="choropleth" id="choropleth" onClick="visualizationChange(this)">Choropleth Map</label>
                <img src="data:image/svg+xml;utf8;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pgo8IS0tIEdlbmVyYXRvcjogQWRvYmUgSWxsdXN0cmF0b3IgMTkuMC4wLCBTVkcgRXhwb3J0IFBsdWctSW4gLiBTVkcgVmVyc2lvbjogNi4wMCBCdWlsZCAwKSAgLS0+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIiBpZD0iTGF5ZXJfMSIgeD0iMHB4IiB5PSIwcHgiIHZpZXdCb3g9IjAgMCA1MTIuMDAyIDUxMi4wMDIiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDUxMi4wMDIgNTEyLjAwMjsiIHhtbDpzcGFjZT0icHJlc2VydmUiIHdpZHRoPSIzMnB4IiBoZWlnaHQ9IjMycHgiPgo8Zz4KCTxnPgoJCTxwYXRoIGQ9Ik03Mi40MzQsOTcuMDU0Yy0xNi4zMzQsMC0yOS42MjQsMTMuMjktMjkuNjI0LDI5LjYyNGMwLDE2LjMzNSwxMy4yOSwyOS42MjUsMjkuNjI0LDI5LjYyNXMyOS42MjQtMTMuMjksMjkuNjI0LTI5LjYyNSAgICBDMTAyLjA1OCwxMTAuMzQ0LDg4Ljc2OCw5Ny4wNTQsNzIuNDM0LDk3LjA1NHogTTcyLjQzNCwxMzQuOTkxYy00LjU4NCwwLTguMzEzLTMuNzMtOC4zMTMtOC4zMTRzMy43MjktOC4zMTMsOC4zMTMtOC4zMTMgICAgYzQuNTg0LDAsOC4zMTMsMy43MjgsOC4zMTMsOC4zMTNTNzcuMDE3LDEzNC45OTEsNzIuNDM0LDEzNC45OTF6IiBmaWxsPSIjMDAwMDAwIi8+Cgk8L2c+CjwvZz4KPGc+Cgk8Zz4KCQk8cGF0aCBkPSJNNDkxLjgxMywxODcuNjE4di03NS4yMzdjMC0xNC41NjUtMTEuODUzLTI2LjQzLTI2LjQzMi0yNi40NDdsLTMzMy4xNjMtMC4xNjdjLTIuNjAzLTMuNzg2LTUuNTc5LTcuMzU5LTguOTI2LTEwLjY2ICAgIEMxMDkuMzg0LDYxLjM5LDkwLjk3Myw1My45NjYsNzEuNDA5LDU0LjI1NUMzMi4yNjcsNTQuNzk3LDAuMjM0LDg3LjA5MSwwLjAwMSwxMjYuMjQxYy0wLjA2OCwxMS42MDIsMi42NzQsMjMuMTU3LDcuOTMsMzMuNDIgICAgYzAuMTA0LDAuMjA0LDE3LjMyOSwyOS4wODYsMTcuMzI5LDI5LjA4NnYyNDIuMzQ4YzAsMTQuNTc3LDExLjg2LDI2LjQzNywyNi40MzIsMjYuNDM3bDQxMy42NjMsMC4yMjQgICAgYzE0LjU4OSwwLDI2LjQ1OC0xMS44NjUsMjYuNDU4LTI2LjQ0OFYyOTEuMzI0YzAsMCwxMi4xNTMtMjAuMzgxLDEyLjI1Ny0yMC41ODZjNS4xODgtMTAuMTI5LDcuOTMxLTIxLjUzNCw3LjkzMS0zMi45ODIgICAgQzUxMiwyMTguODg3LDUwNC44MzUsMjAxLjE2OSw0OTEuODEzLDE4Ny42MTh6IE00MTguOTU1LDEwNy4yMjJsLTUxLjQ0OCw1MS40NDhsLTUxLjUtNTEuNUw0MTguOTU1LDEwNy4yMjJ6IE0zNTMuOTQzLDE3Mi4yMzMgICAgbC04OS42NjUsODkuNjY1bC0zMi43ODQtMzIuNzg0Yy0wLjAwMi0wLjAwMi0wLjAwMy0wLjAwMy0wLjAwNS0wLjAwNXMtMC4wMDMtMC4wMDMtMC4wMDUtMC4wMDVsLTI0LjU5Mi0yNC41OTJsODkuNjY1LTg5LjY2NCAgICBMMzUzLjk0MywxNzIuMjMzeiBNMTc1LjM4NywyOTguNzc3bDQ5LjMyMi00OS4zMjJsNjMuMzY3LDYzLjM2N2wtNDkuMzIyLDQ5LjMyMkwxNzUuMzg3LDI5OC43Nzd6IE0xODcuNDA5LDMzNy45MjMgICAgbC01NS4zMjcsNTUuMzI2bC00NC4zMzItNDQuMzMzbDU1LjMyNi01NS4zMjZMMTg3LjQwOSwzMzcuOTIzeiBNMTM2LjkzMiwxNTkuNjYxYzUuMTg4LTEwLjEyOSw3LjkzMS0yMS41MzQsNy45MzEtMzIuOTgyICAgIGMwLTYuNzI1LTAuOTI4LTEzLjI5OC0yLjY4Ni0xOS41OTZsMTM0Ljk1MywwLjA2OGwtODMuNzk5LDgzLjc5OWwtMzAuMDA3LTMwLjAwN2MtMy43NDYtMy43NDQtOS44MTctMy43NDQtMTMuNTYzLDAgICAgYy0zLjc0NSwzLjc0NS0zLjc0NSw5LjgxOCwwLDEzLjU2NGw2MS4zODYsNjEuMzg2bC00OS4zMjIsNDkuMzIybC0zOS45NDYtMzkuOTQ2Yy0zLjc0NS0zLjc0NC05LjgxNy0zLjc0NC0xMy41NjMsMCAgICBjLTMuNzQ1LDMuNzQ2LTMuNzQ1LDkuODE4LDAsMTMuNTY0bDIxLjE5NiwyMS4xOTZsLTgyLjk0LDgyLjkzOVYyMjQuNDY1bDcuNTY4LDEyLjY4NGMzLjg4MSw2LjUwNCwxMC43MTksMTAuMzg3LDE4LjI5MiwxMC4zODcgICAgczE0LjQxMS0zLjg4MywxOC4yOTItMTAuMzg2QzkwLjcyMywyMzcuMTQ5LDEzNi44MjgsMTU5Ljg2NSwxMzYuOTMyLDE1OS42NjF6IE0yNi43NSwxNDkuNjUyICAgIGMtMy42NTctNy4yNTEtNS40ODctMTUuMDgyLTUuNDM4LTIzLjI4NGMwLjE2NC0yNy42MywyMi43NjktNTAuNDIxLDUwLjM5LTUwLjgwNGMxMy44MTctMC4xNywyNi44MDgsNS4wMzYsMzYuNjI0LDE0LjcxNyAgICBjMy4zOCwzLjMzNCw2LjIyNCw3LjA1OSw4LjUxOCwxMS4wNjVjMC4wOTksMC4xOTEsMC4yMDksMC4zNzQsMC4zMiwwLjU1N2M0LjE2NSw3LjQ5Miw2LjM4OCwxNS45NTYsNi4zODgsMjQuNzc3ICAgIGMwLDguMDkxLTEuODMsMTUuODE4LTUuNDM5LDIyLjk3NEw3Mi40NDMsMjI2LjIzTDI2Ljc1LDE0OS42NTJ6IE01MS42OTgsNDM2LjIyMWMtMi44MjYsMC01LjEyNS0yLjI5OC01LjEyNS01LjEyNXYtNDEuMDAyICAgIGwyNy42MTQtMjcuNjE0bDczLjc5Myw3My43OTRMNTEuNjk4LDQzNi4yMjF6IE0xNzUuMTIsNDM2LjI4N2wtMjkuNDc2LTI5LjQ3Nmw1NS4zMjctNTUuMzI2bDg0Ljg2MSw4NC44NjJMMTc1LjEyLDQzNi4yODd6ICAgICBNMjUyLjMxNywzNzUuNzA2bDQ5LjMyMi00OS4zMjJsMzUuODE2LDM1LjgxNmwtNDkuMzIyLDQ5LjMyMkwyNTIuMzE3LDM3NS43MDZ6IE0zMTIuOTcyLDQzNi4zNjJsLTExLjI3Ny0xMS4yNzdsNDkuMzIyLTQ5LjMyMiAgICBsNjAuNjUzLDYwLjY1M0wzMTIuOTcyLDQzNi4zNjJ6IE00NjUuMzYxLDQzNi40NDRsLTI2LjU1LTAuMDE0bC03NC4yMy03NC4yM2wyMi4zNDEtMjIuMzQxYzMuNzQ2LTMuNzQ2LDMuNzQ2LTkuODE4LDAtMTMuNTY0ICAgIGMtMy43NDUtMy43NDQtOS44MTctMy43NDQtMTMuNTYzLDBsLTIyLjM0MiwyMi4zNDJsLTQyLjU5Mi00Mi41OTJjLTAuMDAyLTAuMDAyLTAuMDAzLTAuMDAzLTAuMDA1LTAuMDA1ICAgIGMtMC4wMDItMC4wMDItMC4wMDMtMC4wMDMtMC4wMDUtMC4wMDVsLTMwLjU3My0zMC41NzRsMzYuMzU0LTM2LjM1NGwxOC44MzgsMTguODM4YzIuMDgxLDIuMDgsNC44MDgsMy4xMjEsNy41MzUsMy4xMjEgICAgczUuNDU1LTEuMDQxLDcuNTM1LTMuMTIxYzQuMTYxLTQuMTYxLDQuMTYxLTEwLjkwOCwwLTE1LjA3MWwtMTguODM4LTE4LjgzOGwxMTYuOC0xMTYuOGwxOS4yOTcsMC4wMSAgICBjMi44MzIsMC4wMDMsNS4xMzcsMi4zMDcsNS4xMzcsNS4xMzV2NTkuODQxYy05Ljg0Ny00LjY2My0yMC43MDYtNy4wNTgtMzEuOTU2LTYuODg4Yy0zOS4xNDIsMC41NDMtNzEuMTc0LDMyLjgzNS03MS40MDYsNzEuOTg2ICAgIGMtMC4wNjgsMTEuNjAyLDIuNjc0LDIzLjE1Nyw3LjkzLDMzLjQyYzAuMTA0LDAuMjAzLDQ2LjIwOSw3Ny40ODgsNDYuMjA5LDc3LjQ4OGMzLjg4MSw2LjUwNCwxMC43MTksMTAuMzg3LDE4LjI5MywxMC4zODcgICAgYzcuNTczLDAsMTQuNDExLTMuODgyLDE4LjI5MS0xMC4zODZsMTIuNjQtMjEuMTg1djEwNC4yNjVoMC4wMDNDNDcwLjUwMSw0MzQuMTQsNDY4LjE5Miw0MzYuNDQ0LDQ2NS4zNjEsNDM2LjQ0NHogTTQ4NS4yNSwyNjAuNzMgICAgbC0xMy40NDksMjIuNTVjLTAuMDA0LDAuMDA5LTAuMDExLDAuMDE3LTAuMDE1LDAuMDI2bC0zMi4yMDcsNTQuMDAxbC00NS42OTEtNzYuNTc4Yy0zLjY1Ny03LjI1MS01LjQ4Ny0xNS4wODItNS40MzgtMjMuMjg0ICAgIGMwLjE2My0yNy42MywyMi43NjktNTAuNDIsNTAuMzkxLTUwLjgwM2MxMy43OS0wLjE4OCwyNi44MDcsNS4wMzUsMzYuNjIyLDE0LjcxNmM5LjgxOCw5LjY4MywxNS4yMjYsMjIuNjEsMTUuMjI2LDM2LjM5OSAgICBDNDkwLjY4OSwyNDUuODQ4LDQ4OC44NTksMjUzLjU3NSw0ODUuMjUsMjYwLjczeiIgZmlsbD0iIzAwMDAwMCIvPgoJPC9nPgo8L2c+CjxnPgoJPGc+CgkJPHBhdGggZD0iTTQzOS41NzEsMjA4LjEzM2MtMTYuMzM1LDAtMjkuNjI0LDEzLjI5LTI5LjYyNCwyOS42MjVjMCwxNi4zMzQsMTMuMjksMjkuNjI0LDI5LjYyNCwyOS42MjQgICAgYzE2LjMzNSwwLDI5LjYyNC0xMy4yOSwyOS42MjQtMjkuNjI0QzQ2OS4xOTUsMjIxLjQyMSw0NTUuOTA2LDIwOC4xMzMsNDM5LjU3MSwyMDguMTMzeiBNNDM5LjU3MSwyNDYuMDY5ICAgIGMtNC41ODQsMC04LjMxMy0zLjcyOC04LjMxMy04LjMxM3MzLjcyOC04LjMxNCw4LjMxMy04LjMxNHM4LjMxMywzLjczLDguMzEzLDguMzE0UzQ0NC4xNTUsMjQ2LjA2OSw0MzkuNTcxLDI0Ni4wNjl6IiBmaWxsPSIjMDAwMDAwIi8+Cgk8L2c+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPGc+CjwvZz4KPC9zdmc+Cg==" />
            </div>
        </div>        
    </div>
    
    </br>
    
    <button class="btn btn-primary btn-md" onclick="Jupyter.notebook.execute_cells([5])">Apply</button>
</div>

In [ ]:
%%local
visualization_type='heatmap'

In [ ]:
%%local
try:
    x_axis_name = widgets.Text(description='X-Axis Title (Optional):', style={'description_width': 'initial'}) 
    y_axis_name = widgets.Text(description='Y-Axis Title (Optional):', style={'description_width': 'initial'}) 
    plot_title = widgets.Text(description='Title (Optional):', style={'description_width': 'initial'})
    plot_subtitle = widgets.Text(description='Subtitle (Optional):', style={'description_width': 'initial'})
    dashboard_picker = widgets.IntText(min=1, max=5, value=1, description='Dashboard Position:', style={'description_width': 'initial'}, layout={'width': '180px'})
    class_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Class Variable (Optional):', style={'description_width': 'initial'})      

    second_part = widgets.VBox([x_axis_name, y_axis_name, plot_title, plot_subtitle])

    if visualization_type == 'scatter' or visualization_type == 'line' or visualization_type == 'area':
        x_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='X-axis Variable:', style={'description_width': 'initial'})
        y_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Y-axis Variable:', style={'description_width': 'initial'})
        x_axis_as_category_var = widgets.Checkbox(value=False, description='X-axis as Category', style={'description_width': 'initial'})
        visualize_button = widgets.Button(description='Visualize', button_style="primary")
        visualize_button.on_click(parameters_button_clicked)

        first_part = widgets.VBox([x_axis_var, x_axis_as_category_var, y_axis_var, class_var, visualize_button])

        display(widgets.HBox([first_part, second_part, dashboard_picker]))

    elif visualization_type == 'pie' or visualization_type == 'bar':
        visualize_button = widgets.Button(description='Visualize', button_style="primary")
        visualize_button.on_click(parameters_button_clicked)
        show_in_legend = widgets.Checkbox(value=False, description='Show Categories in Legend', style={'description_width': 'initial'})
        class_var.description = 'Class Variable'

        if visualization_type == 'pie':
            first_part = widgets.VBox([class_var, show_in_legend, visualize_button])
        else:
            stacked_by = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Stacked By (Optional):', style={'description_width': 'initial'})      
            first_part = widgets.VBox([class_var, stacked_by, visualize_button])

        display(widgets.HBox([first_part, second_part, dashboard_picker]))

    elif visualization_type == 'boxplot':
        x_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Numeric Variable:', style={'description_width': 'initial'})
        visualize_button = widgets.Button(description='Visualize', button_style="primary")
        visualize_button.on_click(parameters_button_clicked)

        first_part = widgets.VBox([x_axis_var, class_var, visualize_button])

        display(widgets.HBox([first_part, second_part, dashboard_picker]))

    elif visualization_type == 'histogram':
        x_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Numeric Variable:', style={'description_width': 'initial'})
        histogram_bins = widgets.IntText(value=5, description='Number of Bins:', style={'description_width': 'initial'})
        visualize_button = widgets.Button(description='Visualize', button_style="primary")
        visualize_button.on_click(parameters_button_clicked)

        first_part = widgets.VBox([x_axis_var, histogram_bins, visualize_button])

        display(widgets.HBox([first_part, second_part, dashboard_picker]))

    elif visualization_type == 'bubble':
        x_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='X-axis Variable:', style={'description_width': 'initial'})
        y_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Y-axis Variable:', style={'description_width': 'initial'})
        z_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Z-axis Variable:', style={'description_width': 'initial'})
        visualize_button = widgets.Button(description='Visualize', button_style="primary")
        visualize_button.on_click(parameters_button_clicked)

        first_part = widgets.VBox([x_axis_var, y_axis_var, z_axis_var, class_var, visualize_button])

        display(widgets.HBox([first_part, second_part, dashboard_picker]))

    elif visualization_type == 'heatmap':
        x_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='X-axis Variable:', style={'description_width': 'initial'})
        y_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Y-axis Variable:', style={'description_width': 'initial'})
        z_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Z-axis Variable:', style={'description_width': 'initial'})
        aggregation_function = widgets.Dropdown(options=['max', 'min', 'sum', 'average', 'count'], value=None, description='Aggregation Function:', style={'description_width': 'initial'})      
        visualize_button = widgets.Button(description='Visualize', button_style="primary")
        visualize_button.on_click(parameters_button_clicked)

        first_part = widgets.VBox([x_axis_var, y_axis_var, z_axis_var, aggregation_function, visualize_button])

        display(widgets.HBox([first_part, second_part, dashboard_picker]))

    elif visualization_type == 'time':
        x_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='X-axis Variable:', style={'description_width': 'initial'})
        x_axis_as_timestamp_var = widgets.Checkbox(value=False, description='Convert X-axis to UNIX timestamp', style={'description_width': 'initial'})
        y_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Y-axis Variable:', style={'description_width': 'initial'})
        visualize_button = widgets.Button(description='Visualize', button_style="primary")
        visualize_button.on_click(parameters_button_clicked)

        first_part = widgets.VBox([x_axis_var, x_axis_as_timestamp_var, y_axis_var, class_var, visualize_button])

        display(widgets.HBox([first_part, second_part, dashboard_picker]))

    elif visualization_type == 'map':
        latitude_var = widgets.Dropdown(options=[x for x in df.columns if x not in ["class"]], value=None, description='Latitude Variable:', style={'description_width': 'initial'})
        longitude_var = widgets.Dropdown(options=[x for x in df.columns if x not in ["class"]], value=None, description='Longitude Variable:', style={'description_width': 'initial'})
        map_type_var = widgets.Dropdown(options=['Markers', 'MarkerCluster', 'FastMarkerCluster', 'Route', 'Route & Markers', 'Heatmap'], value=None, description='Display Type:', layout={'width': '200px'}, style={'description_width': 'initial'})

        markers_colors = widgets.Dropdown(options=[x for x in df.columns if x not in ["class"]], value=None, description='Marker Color (Optional, for Markers only):', layout={'width': '450px'}, style={'description_width': 'initial'})
        markers_text = widgets.Dropdown(options=[x for x in df.columns if x not in ["class"]], value=None, description='Marker Text (Optional, for Markers only):', layout={'width': '450px'}, style={'description_width': 'initial'})
        heatmap_density_var = widgets.Dropdown(options=[x for x in df.columns if x not in ["class"]], value=None, description='Density Variable (Optional, for Heatmap only):', layout={'width': '450px'}, style={'description_width': 'initial'})
        heatmap_opacity_var = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='Point Opacity (Optional, for Heatmap only):', layout={'width': '450px'}, style={'description_width': 'initial'})
        heatmap_point_radius_var = widgets.IntSlider(value=15, min=10, max=30, step=1, description='Heatmap Point Radius (Optional, for Heatmap only):', layout={'width': '450px'}, style={'description_width': 'initial'})
        row_category = widgets.Dropdown(options=[x for x in df.columns if x not in ["class"]], value=None, description='Point Category (Optional, for Routes & Markers option only):', layout={'width': '450px'}, style={'description_width': 'initial'})

        visualize_button = widgets.Button(description='Visualize', button_style="primary")
        visualize_button.on_click(parameters_button_clicked)

        display(widgets.HBox([
            widgets.VBox([latitude_var, longitude_var, map_type_var, dashboard_picker, visualize_button]),
            widgets.VBox([markers_colors, markers_text, heatmap_density_var, heatmap_opacity_var, heatmap_point_radius_var, row_category])
        ]))
        
    elif visualization_type == 'grid':
        visualize_button = widgets.Button(description='Visualize', button_style="primary")
        visualize_button.on_click(parameters_button_clicked)
        
        display(widgets.VBox([dashboard_picker, visualize_button]))
        
    elif visualization_type == 'spline':
        x_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='X-axis Variable:', style={'description_width': 'initial'})
        y_axis_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Y-axis Variable:', style={'description_width': 'initial'})
        x_axis_inverted = widgets.Dropdown(options=['True', 'False'], value='True', description='Invert X-Axis:', style={'description_width': 'initial'})
        visualize_button = widgets.Button(description='Visualize', button_style="primary")
        visualize_button.on_click(parameters_button_clicked)

        first_part = widgets.VBox([x_axis_var, x_axis_inverted, y_axis_var, visualize_button])

        display(widgets.HBox([first_part, second_part, dashboard_picker]))
    
    elif visualization_type == 'choropleth':
        geo_dir = [x for x in get_directory_contents() if '.json' in x]
        
        geo_json_dropdown_var = widgets.Dropdown(options=geo_dir, value=None, description='GEO JSON File:', style={'description_width': 'initial'})
        choropleth_colorscale_var = widgets.Dropdown(options=['BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd'], value=None, description='Color Palette:', style={'description_width': 'initial'})
        choropleth_fill_opacity_var = widgets.FloatSlider(value=0.7, min=0.0, max=1.0, step=0.05, description='Fill Opacity:', layout={'width': '250px'}, style={'description_width': 'initial'})
        choropleth_line_opacity_var = widgets.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.05, description='Line Opacity:', layout={'width': '250px'}, style={'description_width': 'initial'})
        choropleth_legend_var = widgets.Text(description='Legend Text:', style={'description_width': 'initial'}) 
        choropleth_key_column_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Data Key Column:', style={'description_width': 'initial'})
        choropleth_value_column_var = widgets.Dropdown(options=[x for x in df.columns], value=None, description='Data Value Column:', style={'description_width': 'initial'})
        choropleth_join_column_var = widgets.Text(description='GEO-JSON Join Column (e.g. feature.id):', layout={'width': '450px'}, style={'description_width': 'initial'}) 
        
        visualize_button = widgets.Button(description='Visualize', button_style="primary")
        visualize_button.on_click(parameters_button_clicked)
        
        first_part = widgets.VBox([geo_json_dropdown_var, choropleth_colorscale_var, choropleth_fill_opacity_var, choropleth_line_opacity_var, visualize_button])
        second_part = widgets.VBox([choropleth_legend_var, choropleth_key_column_var, choropleth_value_column_var, choropleth_join_column_var])
        
        display(widgets.HBox([first_part, second_part, dashboard_picker]))
        
except:
    print(colored("An exception occured while trying to create parameter list.\n Please make sure that you have correctly openned a CSV file.", "red", attrs=['bold']))

In [ ]:
%%local
chart1 = create_chart(df, visualization_type)

if chart1 is not None:
    display(chart1)
    
    def save_chart1_local(b):
        success = save_chart(chart1, save_chart1_name.value)
        if (success):
            save_chart1_label.layout.visibility="visible"
            save_chart1_label.value = "File successfully stored"
        else:
            save_chart1_label.layout.visibility="visible"
            save_chart1_label.value = "An error occured while saving the file"
    
    save_chart1_button = widgets.Button(description='Save Chart as HTML', button_style="primary")
    save_chart1_name = widgets.Text(value='Chart',placeholder='Chart',description='File Name:',disabled=False)
    save_chart1_label = widgets.Label(value="", layout={"margin": "0px 0px 0px 30px", "visibility": "hidden"})
    save_chart1_hbox = widgets.HBox([save_chart1_name, save_chart1_button, save_chart1_label])

    save_chart1_button.on_click(save_chart1_local)

    if 'grid' not in str(type(chart1)):
        display(save_chart1_hbox)

In [ ]:
%%local
chart2 = create_chart(df, visualization_type)

if chart2 is not None:
    display(chart2)
    
    def save_chart2_local(b):
        success = save_chart(chart2, save_chart2_name.value)
        if (success):
            save_chart2_label.layout.visibility="visible"
            save_chart2_label.value = "File successfully stored"
        else:
            save_chart2_label.layout.visibility="visible"
            save_chart2_label.value = "An error occured while saving the file"
    
    save_chart2_button = widgets.Button(description='Save Chart as HTML', button_style="primary")
    save_chart2_name = widgets.Text(value='Chart',placeholder='Chart',description='File Name:',disabled=False)
    save_chart2_label = widgets.Label(value="", layout={"margin": "0px 0px 0px 30px", "visibility": "hidden"})
    save_chart2_hbox = widgets.HBox([save_chart2_name, save_chart2_button, save_chart2_label])

    save_chart2_button.on_click(save_chart2_local)

    if 'grid' not in str(type(chart2)):
        display(save_chart2_hbox)

In [ ]:
%%local
chart3 = create_chart(df, visualization_type)

if chart3 is not None:
    display(chart3)
    
    def save_chart3_local(b):
        success = save_chart(chart3, save_chart3_name.value)
        if (success):
            save_chart3_label.layout.visibility="visible"
            save_chart3_label.value = "File successfully stored"
        else:
            save_chart3_label.layout.visibility="visible"
            save_chart3_label.value = "An error occured while saving the file"
    
    save_chart3_button = widgets.Button(description='Save Chart as HTML', button_style="primary")
    save_chart3_name = widgets.Text(value='Chart',placeholder='Chart',description='File Name:',disabled=False)
    save_chart3_label = widgets.Label(value="", layout={"margin": "0px 0px 0px 30px", "visibility": "hidden"})
    save_chart3_hbox = widgets.HBox([save_chart3_name, save_chart3_button, save_chart3_label])

    save_chart3_button.on_click(save_chart3_local)

    if 'grid' not in str(type(chart3)):
        display(save_chart3_hbox)

In [ ]:
%%local
chart4 = create_chart(df, visualization_type)

if chart4 is not None:
    display(chart4)
    
    def save_chart4_local(b):
        success = save_chart(chart4, save_chart4_name.value)
        if (success):
            save_chart4_label.layout.visibility="visible"
            save_chart4_label.value = "File successfully stored"
        else:
            save_chart4_label.layout.visibility="visible"
            save_chart4_label.value = "An error occured while saving the file"
    
    save_chart4_button = widgets.Button(description='Save Chart as HTML', button_style="primary")
    save_chart4_name = widgets.Text(value='Chart',placeholder='Chart',description='File Name:',disabled=False)
    save_chart4_label = widgets.Label(value="", layout={"margin": "0px 0px 0px 30px", "visibility": "hidden"})
    save_chart4_hbox = widgets.HBox([save_chart4_name, save_chart4_button, save_chart4_label])

    save_chart4_button.on_click(save_chart4_local)

    if 'grid' not in str(type(chart4)):
        display(save_chart4_hbox)

In [ ]:
%%local
chart5 = create_chart(df, visualization_type)

if chart5 is not None:
    display(chart5)
    
    def save_chart5_local(b):
        success = save_chart(chart5, save_chart5_name.value)
        if (success):
            save_chart5_label.layout.visibility="visible"
            save_chart5_label.value = "File successfully stored"
        else:
            save_chart5_label.layout.visibility="visible"
            save_chart5_label.value = "An error occured while saving the file"
    
    save_chart5_button = widgets.Button(description='Save Chart as HTML', button_style="primary")
    save_chart5_name = widgets.Text(value='Chart',placeholder='Chart',description='File Name:',disabled=False)
    save_chart5_label = widgets.Label(value="", layout={"margin": "0px 0px 0px 30px", "visibility": "hidden"})
    save_chart5_hbox = widgets.HBox([save_chart5_name, save_chart5_button, save_chart5_label])

    save_chart5_button.on_click(save_chart5_local)

    if 'grid' not in str(type(chart5)):
        display(save_chart5_hbox)